<a href="https://colab.research.google.com/github/rjshvjy/tds-automation/blob/main/TDS_Automation_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Install required libraries and import modules
# This cell installs necessary packages and imports all required libraries

# Install required packages (run this only once)
!pip install PyPDF2 openpyxl pandas numpy tabulate

# Import all necessary libraries
import PyPDF2
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")
print("Ready to process TDS files...")
print("\n📋 Expected file naming conventions:")
print("   - Masters file: TDS_Masters*.xlsx")
print("   - Template file: TDS_Template*.xlsx")
print("   - Output file: TDS_[Month]_[Year].xlsx (auto-generated from payment dates)")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00
✅ All libraries imported successfully!
Ready to process TDS files...

📋 Expected file naming conventions:
   - Masters file: TDS_Masters*.xlsx
   - Template file: TDS_Template*.xlsx
   - Output file: TDS_[Month]_[Year].xlsx (auto-generated from payment dates)


In [2]:
# Cell 2: Functions to extract data from PDF challans - FIXED AMOUNT EXTRACTION
# This cell contains all the functions needed to read and parse PDF challan files

def extract_challan_data_from_pdf(pdf_path):
    """
    Extract challan data from a single PDF file
    Returns a dictionary with all challan details
    """
    challan_data = {}

    try:
        # Open and read the PDF
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)

            # Extract text from first page
            text = ""
            if len(pdf_reader.pages) > 0:
                text = pdf_reader.pages[0].extract_text()

            # Extract all required fields using regex patterns - IMPROVED PATTERNS
            patterns = {
                'tan': r'TAN\s*:\s*([A-Z0-9]+)',
                'nature_of_payment': r'Nature of Payment\s*:\s*(\d+[A-Z])',
                'cin': r'CIN\s*:\s*([A-Z0-9]+)',
                'bsr_code': r'BSR code\s*:\s*([\d]+)',
                'challan_no': r'Challan No\s*:\s*([\d]+)',
                'tender_date': r'Tender Date\s*:\s*(\d{2}/\d{2}/\d{4})',
                'mode_of_payment': r'Mode of Payment\s*:\s*([^\n]+)',
            }

            # Extract each field
            for field, pattern in patterns.items():
                match = re.search(pattern, text, re.IGNORECASE)
                if match:
                    value = match.group(1).strip()
                    # Convert mode of payment to uppercase
                    if field == 'mode_of_payment':
                        value = value.upper()
                    # Keep BSR code and challan_no as strings to preserve leading zeros
                    elif field in ['bsr_code', 'challan_no']:
                        value = value.zfill(7) if field == 'bsr_code' else value
                    challan_data[field] = value
                else:
                    challan_data[field] = ""

            # SPECIAL HANDLING FOR AMOUNTS - Multiple patterns to try
            # Pattern 1: Try the tax breakup section with flexible whitespace
            tax_patterns = [
                r'A\s+Tax\s+₹\s*([\d,]+)',  # Original pattern
                r'A\s+Tax\s+₹\s*([\d,]+)',  # With regular space
                r'A\s+Tax\s+[₹]\s*([\d,]+)',  # ₹ in brackets
                r'A\s+Tax\s+.\s*([\d,]+)',  # Any character instead of ₹
                r'Tax\s+₹\s*([\d,]+)',  # Simplified pattern
                r'A\s+Tax[^0-9]+([\d,]+)',  # Skip any non-digits after Tax
            ]

            tax_amount = ""
            for pattern in tax_patterns:
                match = re.search(pattern, text, re.IGNORECASE)
                if match:
                    tax_amount = match.group(1).strip().replace(',', '')
                    break

            # Fallback: Try the header amount field
            if not tax_amount:
                amount_patterns = [
                    r'Amount \(in Rs\.\)\s*:\s*₹\s*([\d,]+)',
                    r'Amount.*?₹\s*([\d,]+)',
                    r'Amount.*?Rs.*?([\d,]+)',
                ]
                for pattern in amount_patterns:
                    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
                    if match:
                        tax_amount = match.group(1).strip().replace(',', '')
                        break

            challan_data['tax_amount'] = tax_amount

            # Extract other amounts with similar flexible patterns
            amount_fields = {
                'surcharge': [
                    r'B\s+Surcharge\s+₹\s*([\d,]+)',
                    r'B\s+Surcharge[^0-9]+([\d,]+)',
                    r'Surcharge\s+₹\s*([\d,]+)'
                ],
                'cess': [
                    r'C\s+Cess\s+₹\s*([\d,]+)',
                    r'C\s+Cess[^0-9]+([\d,]+)',
                    r'Cess\s+₹\s*([\d,]+)'
                ],
                'interest': [
                    r'D\s+Interest\s+₹\s*([\d,]+)',
                    r'D\s+Interest[^0-9]+([\d,]+)',
                    r'Interest\s+₹\s*([\d,]+)'
                ],
                'penalty': [
                    r'E\s+Penalty\s+₹\s*([\d,]+)',
                    r'E\s+Penalty[^0-9]+([\d,]+)',
                    r'Penalty\s+₹\s*([\d,]+)'
                ],
                'fee_234e': [
                    r'F\s+Fee under section 234E\s+₹\s*([\d,]+)',
                    r'Fee under section 234E\s+₹\s*([\d,]+)',
                    r'234E[^0-9]+([\d,]+)'
                ]
            }

            for field, patterns_list in amount_fields.items():
                value = ""
                for pattern in patterns_list:
                    match = re.search(pattern, text, re.IGNORECASE)
                    if match:
                        value = match.group(1).strip().replace(',', '')
                        break
                challan_data[field] = value if value else "0"

            # Extract total amount
            total_patterns = [
                r'Total \(A\+B\+C\+D\+E\+F\)\s+₹\s*([\d,]+)',
                r'Total.*?₹\s*([\d,]+)',
                r'Total[^0-9]+([\d,]+)'
            ]

            total_amount = ""
            for pattern in total_patterns:
                match = re.search(pattern, text, re.IGNORECASE)
                if match:
                    total_amount = match.group(1).strip().replace(',', '')
                    break

            challan_data['total_amount'] = total_amount

            # Add file name for reference
            challan_data['file_name'] = os.path.basename(pdf_path)

            print(f"✅ Extracted data from: {os.path.basename(pdf_path)}")
            print(f"   Nature of Payment: {challan_data.get('nature_of_payment', 'Not found')}")
            print(f"   Challan No: {challan_data.get('challan_no', 'Not found')}")
            print(f"   Tax Amount: ₹{challan_data.get('tax_amount', 'Not found')}")

            # Debug: Show all amounts if tax amount is found
            if challan_data.get('tax_amount'):
                print(f"   Surcharge: ₹{challan_data.get('surcharge', '0')}")
                print(f"   Cess: ₹{challan_data.get('cess', '0')}")
                print(f"   Total: ₹{challan_data.get('total_amount', 'Not found')}")

    except Exception as e:
        print(f"❌ Error processing {pdf_path}: {str(e)}")
        challan_data['error'] = str(e)

    return challan_data

def extract_all_challans(pdf_folder_path):
    """
    Extract data from all PDF files in a folder and DEDUPLICATE by challan number
    Returns a list of dictionaries, one for each UNIQUE challan
    """
    all_challan_data = []
    challan_map = {}  # To track unique challans by challan number

    # Get all PDF files in the folder
    pdf_files = [f for f in os.listdir(pdf_folder_path) if f.lower().endswith('.pdf')]

    if not pdf_files:
        print(f"❌ No PDF files found in {pdf_folder_path}")
        return all_challan_data

    print(f"\n📁 Found {len(pdf_files)} PDF files to process...")
    print("-" * 50)

    # Process each PDF
    duplicate_count = 0
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder_path, pdf_file)
        challan_data = extract_challan_data_from_pdf(pdf_path)

        # Check if this challan number already exists
        challan_no = challan_data.get('challan_no', '')

        if challan_no and challan_no in challan_map:
            # This is a duplicate challan
            duplicate_count += 1
            existing_challan = challan_map[challan_no]

            # Verify the duplicate has same amount (data integrity check)
            if challan_data.get('tax_amount') != existing_challan.get('tax_amount'):
                print(f"⚠️  WARNING: Duplicate challan {challan_no} has different tax amounts!")
                print(f"   File 1: {existing_challan.get('file_name')} - ₹{existing_challan.get('tax_amount')}")
                print(f"   File 2: {challan_data.get('file_name')} - ₹{challan_data.get('tax_amount')}")
        else:
            # This is a new unique challan
            if challan_no:  # Only add if challan number exists
                challan_map[challan_no] = challan_data
                all_challan_data.append(challan_data)
            else:
                print(f"⚠️  Skipping file {pdf_file} - no challan number found")

    print("-" * 50)
    print(f"✅ Total PDF files processed: {len(pdf_files)}")
    print(f"✅ Unique challans found: {len(all_challan_data)}")
    if duplicate_count > 0:
        print(f"ℹ️  Duplicate challans skipped: {duplicate_count}")

    # Create summary by Nature of Payment
    summary = {}
    total_all = 0
    for challan in all_challan_data:
        nop = challan.get('nature_of_payment', 'Unknown')
        if nop:
            if nop not in summary:
                summary[nop] = {'count': 0, 'total_tax': 0}
            summary[nop]['count'] += 1
            try:
                tax_amt = float(challan.get('tax_amount', 0))
                summary[nop]['total_tax'] += tax_amt
                total_all += tax_amt
            except:
                pass

    print("\n📊 Summary by Nature of Payment:")
    for nop, data in sorted(summary.items()):
        print(f"   {nop}: {data['count']} challan(s), Total Tax: ₹{data['total_tax']:,.0f}")
    print(f"   GRAND TOTAL: ₹{total_all:,.0f}")

    return all_challan_data

# Test function to verify extraction is working
def test_extraction():
    """Test the extraction with a sample text"""
    print("PDF extraction functions loaded successfully!")
    print("Now with:")
    print("  ✅ Improved amount extraction with multiple fallback patterns")
    print("  ✅ Fixed regex patterns for better ₹ symbol handling")
    print("  ✅ Deduplication based on challan number")
    print("  ✅ Data integrity checks for duplicates")
    print("  ✅ Mode of payment converted to uppercase")
    print("  ✅ BSR codes and challan numbers preserved as strings")
    print("Use extract_all_challans('/path/to/pdf/folder') to process your PDF files")

test_extraction()

PDF extraction functions loaded successfully!
Now with:
  ✅ Improved amount extraction with multiple fallback patterns
  ✅ Fixed regex patterns for better ₹ symbol handling
  ✅ Deduplication based on challan number
  ✅ Data integrity checks for duplicates
  ✅ Mode of payment converted to uppercase
  ✅ BSR codes and challan numbers preserved as strings
Use extract_all_challans('/path/to/pdf/folder') to process your PDF files


In [3]:
# Cell 3: Functions to read and update Excel files - FIXED WITH PROPER COLUMN CODES AND TDS PRESERVATION
# This cell handles reading TDS Masters and updating it with challan data

import math
from decimal import Decimal, ROUND_HALF_UP

def read_tds_masters(file_path):
    """
    Read the TDS Masters Excel file and return data from all sheets
    FIXED: Uses openpyxl with data_only=True to read static values, avoiding type inference issues
    """
    try:
        # Use pandas for other sheets
        tds_codes = pd.read_excel(file_path, sheet_name='TDS CODES', keep_default_na=False)
        tds_rates = pd.read_excel(file_path, sheet_name='TDS RATES', keep_default_na=False)

        # Use openpyxl for TDS PARTIES to preserve values
        wb = load_workbook(file_path, data_only=True)
        ws_parties = wb['TDS PARTIES']

        # Find the row with column codes
        code_row = None
        for idx in range(1, 11):  # Check first 10 rows
            row_values = []
            for col in range(1, ws_parties.max_column + 1):
                cell_value = ws_parties.cell(row=idx, column=col).value
                if cell_value:
                    row_values.append(str(cell_value))

            code_patterns = ['(415)', '(427)', '(416)', '(415A)', '-415', '-427', '-416', '-417', '-418', '-419', '-421']
            if any(pattern in val for val in row_values for pattern in code_patterns):
                code_row = idx
                print(f"✅ Found column codes at row {idx}")
                break

        # Read headers (row before codes)
        header_row = code_row - 1 if code_row else 1
        headers = []
        for col in range(1, ws_parties.max_column + 1):
            header_val = ws_parties.cell(row=header_row, column=col).value
            headers.append(header_val if header_val else f"Column_{col}")

        # Get column codes
        code_to_column_name = {}
        column_code_map = {}
        if code_row:
            for col in range(1, ws_parties.max_column + 1):
                code_val = ws_parties.cell(row=code_row, column=col).value
                if code_val:
                    code_str = str(code_val).strip()
                    code_match = None
                    if '(' in code_str and ')' in code_str:
                        code_match = re.search(r'\\(([0-9A-Z]+)\\)', code_str)
                    elif code_str.startswith('-'):
                        code_match = re.search(r'-([0-9A-Z]+)', code_str)
                    if code_match:
                        extracted_code = code_match.group(1)
                        normalized_code = f'({extracted_code})'
                        column_code_map[normalized_code] = col - 1
                        code_to_column_name[normalized_code] = headers[col - 1]

        # Fallback mappings by column names
        column_name_mappings = {
            '(415)': ['Deductee Code', 'Individual/Company', 'Indiv/Comp', 'Code'],
            '(415A)': ['Section Under Payment Made', 'Type of Payment', 'Nature of Payment', 'Section'],
            '(416)': ['PAN of the Deductee', 'PAN', 'PAN No', 'Deductee PAN'],
            '(417)': ['Name of the Deductee', 'Deductee Name', 'Name', 'Party Name'],
            '(418)': ['Date of Payment/credit', 'Payment Date', 'Date of Payment', 'Credit Date'],
            '(419)': ['Amount Paid /Credited', 'Amount Paid', 'Gross Amount', 'Payment Amount', 'Amount'],
            '(421)': ['TDS', 'Tax Deducted', 'TDS Amount', 'TDS               Rs.', 'TDS Rs.'],
            '(425D)': ['BSR Code', 'BSR', 'Bank BSR Code'],
            '(425E)': ['Challan Serial No', 'Challan No', 'Challan Number'],
            '(425F)': ['Date on which deposited', 'Date Deposited', 'Deposit Date', 'Challan Date'],
            '(427)': ['TDS Deducted Rates %', 'TDS Rate', 'Rate %', 'Deduction Rate', 'Rate']
        }

        for code, possible_names in column_name_mappings.items():
            if code not in code_to_column_name:
                for col_idx, col_name in enumerate(headers):
                    col_name_clean = str(col_name).strip()
                    for possible_name in possible_names:
                        if possible_name.lower() in col_name_clean.lower():
                            code_to_column_name[code] = col_name
                            column_code_map[code] = col_idx
                            print(f"   Found {code} by column name: '{col_name}'")
                            break
                    if code in code_to_column_name:
                        break

        # Read data rows
        data_rows = []
        data_start_row = code_row + 1 if code_row else 2
        for row in range(data_start_row, ws_parties.max_row + 1):
            row_data = []
            is_empty_row = True
            for col in range(1, ws_parties.max_column + 1):
                cell_value = ws_parties.cell(row=row, column=col).value
                row_data.append(cell_value)
                if cell_value is not None and str(cell_value).strip():
                    is_empty_row = False
            if not is_empty_row:
                data_rows.append(row_data)

        tds_parties = pd.DataFrame(data_rows, columns=headers)

        # Convert numeric columns with precise rounding - EXCLUDE (427) TO PRESERVE DECIMAL RATES
        numeric_codes = ['(419)', '(421)']  # Removed '(427)' to avoid quantizing rates to integers
        for code in numeric_codes:
            col_name = code_to_column_name.get(code)
            if col_name and col_name in tds_parties.columns:
                tds_parties[col_name] = pd.to_numeric(
                    tds_parties[col_name].astype(str).str.replace(',', '').str.replace('₹', '').str.strip(),
                    errors='coerce'
                ).apply(lambda x: Decimal(str(x)).quantize(Decimal('1.'), rounding=ROUND_HALF_UP) if pd.notna(x) else x)
                print(f"   ✅ Converted '{col_name}' to numeric with ROUND_HALF_UP")

        # Convert date columns
        date_codes = ['(418)', '(425F)']
        for code in date_codes:
            col_name = code_to_column_name.get(code)
            if col_name and col_name in tds_parties.columns:
                tds_parties[col_name] = pd.to_datetime(tds_parties[col_name], errors='coerce', dayfirst=True)
                print(f"   ✅ Converted '{col_name}' to datetime")

        # Validate PANs
        pan_col = code_to_column_name.get('(416)')
        if pan_col and pan_col in tds_parties.columns:
            for idx, pan in tds_parties[pan_col].items():
                if pd.notna(pan) and not re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$', str(pan)):
                    print(f"⚠️ Invalid PAN format at row {idx + data_start_row}: {pan}")

        challan_details = pd.read_excel(file_path, sheet_name='Challan Details', header=1, keep_default_na=False)
        wb.close()

        print(f"\\n✅ Successfully read TDS Masters file")
        print(f"   TDS PARTIES: {len(tds_parties)} rows")
        print(f"   Challan Details: {len(challan_details)} rows")
        print(f"   Column codes mapped: {len(code_to_column_name)}")

        tds_col = code_to_column_name.get('(421)', None)
        if tds_col and tds_col in tds_parties.columns:
            print(f"\\n📊 Sample TDS amounts from first 5 rows:")
            for idx in range(min(5, len(tds_parties))):
                payment_type = tds_parties.iloc[idx].get(code_to_column_name.get('(415A)', ''), '')
                tds_amount = tds_parties.iloc[idx].get(tds_col, 0)
                name = tds_parties.iloc[idx].get(code_to_column_name.get('(417)', ''), '')
                print(f"   Row {idx}: Name={name}, Payment={payment_type}, TDS={tds_amount}")

        print("\\n📊 Column Code Mapping Found:")
        for code, col_name in sorted(code_to_column_name.items())[:15]:
            print(f"   Code {code} → Column: '{col_name}'")

        return {
            'tds_codes': tds_codes,
            'tds_parties': tds_parties,
            'challan_details': challan_details,
            'tds_rates': tds_rates,
            'file_path': file_path,
            'column_code_map': column_code_map,
            'code_to_column_name': code_to_column_name,
            'code_row': code_row
        }

    except Exception as e:
        print(f"❌ Error reading TDS Masters: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

def update_tds_masters_with_challans(tds_masters_data, challan_data_list):
    """
    Update TDS Masters with challan information
    FIXED: Uses data_only=True to preserve static TDS values, writes BSR/challan as strings
    """
    try:
        # Load workbook with data_only=True to preserve static values
        wb = load_workbook(tds_masters_data['file_path'], data_only=True)
        ws_parties = wb['TDS PARTIES']
        ws_challan = wb['Challan Details']

        code_to_column_name = tds_masters_data.get('code_to_column_name', {})
        code_row = tds_masters_data.get('code_row', 1)

        # Find columns by codes
        col_425E = col_425F = col_415A = None
        for col_idx in range(1, ws_parties.max_column + 1):
            cell_value = str(ws_parties.cell(row=code_row, column=col_idx).value)
            if '425E' in cell_value:
                col_425E = col_idx
                print(f"Found (425E) at column {col_idx}")
            elif '425F' in cell_value:
                col_425F = col_idx
                print(f"Found (425F) at column {col_idx}")
            elif '415A' in cell_value:
                col_415A = col_idx
                print(f"Found (415A) (Type of Payment) at column {col_idx}")

        # Create mapping of nature of payment to challan data
        challan_map = {}
        for challan in challan_data_list:
            nop = challan.get('nature_of_payment', '')
            if nop:
                nop_clean = nop.replace(' ', '')
                challan_map[nop_clean] = challan

        print(f"\\n📝 Updating TDS PARTIES sheet...")
        print(f"   Challan Serial No (425E) → Column {col_425E}")
        print(f"   Date deposited (425F) → Column {col_425F}")
        updates_made = 0

        # Update TDS PARTIES
        data_start_row = code_row + 1
        for row_idx in range(data_start_row, ws_parties.max_row + 1):
            payment_type = ws_parties.cell(row=row_idx, column=col_415A).value if col_415A else None
            if payment_type and str(payment_type).strip() not in ['', 'nan', 'None']:
                payment_type_clean = str(payment_type).replace(' ', '').strip()
                if payment_type_clean in challan_map:
                    challan = challan_map[payment_type_clean]
                    if col_425E:
                        ws_parties.cell(row=row_idx, column=col_425E).value = challan.get('challan_no', '')
                    if col_425F:
                        date_str = challan.get('tender_date', '')
                        if date_str:
                            try:
                                date_obj = datetime.strptime(date_str, '%d/%m/%Y')
                                ws_parties.cell(row=row_idx, column=col_425F).value = date_obj
                                ws_parties.cell(row=row_idx, column=col_425F).number_format = 'DD/MM/YYYY'
                            except:
                                ws_parties.cell(row=row_idx, column=col_425F).value = date_str
                    updates_made += 1

        print(f"✅ Updated {updates_made} rows in TDS PARTIES")

        # Update Challan Details
        print("\\n📝 Updating Challan Details sheet...")
        for row in ws_challan.iter_rows(min_row=3, max_row=ws_challan.max_row):
            for cell in row:
                cell.value = None

        for idx, challan in enumerate(challan_data_list, start=3):
            tax_amt = Decimal(challan.get('tax_amount', 0)).quantize(Decimal('1.'), rounding=ROUND_HALF_UP) if challan.get('tax_amount', '') else 0
            surcharge = Decimal(challan.get('surcharge', 0)).quantize(Decimal('1.'), rounding=ROUND_HALF_UP) if challan.get('surcharge', '') else 0
            cess = Decimal(challan.get('cess', 0)).quantize(Decimal('1.'), rounding=ROUND_HALF_UP) if challan.get('cess', '') else 0
            interest = Decimal(challan.get('interest', 0)).quantize(Decimal('1.'), rounding=ROUND_HALF_UP) if challan.get('interest', '') else 0
            penalty = Decimal(challan.get('penalty', 0)).quantize(Decimal('1.'), rounding=ROUND_HALF_UP) if challan.get('penalty', '') else 0

            ws_challan.cell(row=idx, column=1).value = idx - 2
            ws_challan.cell(row=idx, column=2).value = challan.get('nature_of_payment', '')
            ws_challan.cell(row=idx, column=3).value = int(tax_amt)
            ws_challan.cell(row=idx, column=4).value = int(surcharge)
            ws_challan.cell(row=idx, column=5).value = int(cess)
            ws_challan.cell(row=idx, column=6).value = int(interest)
            ws_challan.cell(row=idx, column=7).value = int(penalty)
            ws_challan.cell(row=idx, column=8).value = f'=SUM(C{idx}:G{idx})'
            ws_challan.cell(row=idx, column=9).value = challan.get('mode_of_payment', '')
            ws_challan.cell(row=idx, column=10).value = challan.get('bsr_code', '')
            date_str = challan.get('tender_date', '')
            if date_str:
                try:
                    date_obj = datetime.strptime(date_str, '%d/%m/%Y')
                    ws_challan.cell(row=idx, column=11).value = date_obj
                    ws_challan.cell(row=idx, column=11).number_format = 'DD/MM/YYYY'
                except:
                    ws_challan.cell(row=idx, column=11).value = date_str
            ws_challan.cell(row=idx, column=12).value = challan.get('challan_no', '')
            ws_challan.cell(row=idx, column=13).value = 'NO'

        print(f"✅ Added {len(challan_data_list)} challans to Challan Details")

        output_file = tds_masters_data['file_path'].replace('.xlsx', '_UPDATED.xlsx')
        wb.save(output_file)
        wb.close()

        print(f"\\n✅ Saved updated TDS Masters to: {output_file}")
        return read_tds_masters(output_file)

    except Exception as e:
        print(f"❌ Error updating TDS Masters: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

def validate_tds_totals(tds_masters_data, challan_data_list):
    """
    Validate that party-wise TDS totals match challan amounts
    Uses column codes to identify the correct columns
    """
    print("\\n🔍 Validating TDS totals...")

    try:
        tds_parties = tds_masters_data['tds_parties']
        code_to_column_name = tds_masters_data.get('code_to_column_name', {})
        payment_col = code_to_column_name.get('(415A)', None)
        tds_col = code_to_column_name.get('(421)', None)

        print(f"\\n📊 Debug - Column mappings:")
        print(f"   Payment Type column (415A): {payment_col}")
        print(f"   TDS Amount column (421): {tds_col}")

        if not payment_col or not tds_col:
            print("⚠️ Missing required columns")
            return False

        print(f"\\n📊 Debug - Sample data (first 5 valid rows):")
        sample_count = 0
        for idx, row in tds_parties.iterrows():
            if sample_count >= 5:
                break
            payment = row.get(payment_col, '') if payment_col else ''
            tds_amount = row.get(tds_col, 0) if tds_col else 0
            if payment and str(payment) not in ['nan', 'NaT', '']:
                print(f"   Row {idx}: Payment={payment}, TDS={tds_amount}")
                sample_count += 1

        party_totals = {}
        for _, row in tds_parties.iterrows():
            payment_type = str(row[payment_col] if payment_col in row else '').strip()
            if payment_type and payment_type != 'nan' and payment_type != 'NaT':
                tds_amount = 0
                if tds_col and tds_col in row:
                    try:
                        val = row[tds_col]
                        if pd.notna(val):
                            tds_amount = Decimal(str(val)).quantize(Decimal('1.'), rounding=ROUND_HALF_UP)
                    except Exception as e:
                        print(f"   Warning: Could not convert TDS value '{val}' for payment type {payment_type}: {e}")
                        tds_amount = 0
                payment_type_clean = payment_type.replace(' ', '')
                if payment_type_clean not in party_totals:
                    party_totals[payment_type_clean] = 0
                party_totals[payment_type_clean] += tds_amount

        challan_totals = {}
        for challan in challan_data_list:
            nop = challan.get('nature_of_payment', '').replace(' ', '')
            tax_amount = Decimal(challan.get('tax_amount', 0)).quantize(Decimal('1.'), rounding=ROUND_HALF_UP) if challan.get('tax_amount', '') else 0
            if nop:
                challan_totals[nop] = tax_amount

        print(f"\\n📊 Debug - Totals found:")
        print(f"   Party totals: {party_totals}")
        print(f"   Challan totals: {challan_totals}")

        validation_passed = True
        print("\\n📊 Validation Results:")
        print("-" * 60)
        print(f"{'Nature of Payment':<20} {'Party Total':<15} {'Challan Total':<15} {'Status':<10}")
        print("-" * 60)

        for nop in sorted(set(list(party_totals.keys()) + list(challan_totals.keys()))):
            party_total = party_totals.get(nop, 0)
            challan_total = challan_totals.get(nop, 0)
            difference = abs(party_total - challan_total)
            status = "✅ PASS" if difference <= 1 else "❌ FAIL"
            if difference > 1:
                validation_passed = False
            print(f"{nop:<20} ₹{party_total:<14,.0f} ₹{challan_total:<14,.0f} {status}")

        print("-" * 60)
        print("\\n✅ All validations passed!" if validation_passed else "\\n❌ Validation failed! Please check the discrepancies above.")

        return validation_passed

    except Exception as e:
        print(f"❌ Error during validation: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

print("✅ Excel handling functions loaded - FIXED WITH TDS PRESERVATION!")
print("   ✓ Uses openpyxl with data_only=True to preserve static TDS values")
print("   ✓ Precise rounding with Decimal and ROUND_HALF_UP")
print("   ✓ PAN format validation added")
print("   ✓ BSR codes and challan numbers preserved as strings")
print("   ✓ Handles both (code) and -code formats")
print("   ✓ Proper date parsing and formatting")

✅ Excel handling functions loaded - FIXED WITH TDS PRESERVATION!
   ✓ Uses openpyxl with data_only=True to preserve static TDS values
   ✓ Precise rounding with Decimal and ROUND_HALF_UP
   ✓ PAN format validation added
   ✓ BSR codes and challan numbers preserved as strings
   ✓ Handles both (code) and -code formats
   ✓ Proper date parsing and formatting


In [4]:
# Cell 8: TDS_Masters Template Generator
# Creates downloadable blank TDS_Masters template for users to fill offline
# This template becomes the input file for processing

import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.datavalidation import DataValidation
import pandas as pd
from datetime import datetime

class TDSMastersTemplateGenerator:
    """
    Generate blank TDS_Masters.xlsx template for user download
    This template contains all sheets needed for user input
    """

    def __init__(self):
        self.wb = None

    def create_template(self):
        """Create complete TDS_Masters template with all sheets"""
        self.wb = Workbook()

        # Sheet 1: INPUT-DEDUCTOR DETAILS (User fills company info)
        ws_deductor = self.wb.active
        ws_deductor.title = "INPUT-DEDUCTOR DETAILS"
        self._setup_deductor_input_sheet(ws_deductor)

        # Sheet 2: TDS CODES (Reference sheet)
        ws_codes = self.wb.create_sheet("TDS CODES")
        self._setup_tds_codes_sheet(ws_codes)

        # Sheet 3: TDS PARTIES (Main data entry sheet)
        ws_parties = self.wb.create_sheet("TDS PARTIES")
        self._setup_tds_parties_sheet(ws_parties)

        # Sheet 4: Challan Details (System fills from PDFs - blank for user)
        ws_challan = self.wb.create_sheet("Challan Details")
        self._setup_challan_details_sheet(ws_challan)

        # Sheet 5: TDS RATES (Reference sheet)
        ws_rates = self.wb.create_sheet("TDS RATES")
        self._setup_tds_rates_sheet(ws_rates)

        return self.wb

    def _setup_deductor_input_sheet(self, ws):
        """Setup INPUT-DEDUCTOR DETAILS sheet for user input"""

        # Title
        ws['A1'] = "INPUT-DEDUCTOR DETAILS"
        ws['A1'].font = Font(bold=True, size=14, color="FF0000")
        ws.merge_cells('A1:B1')

        # Deductor Information Section
        ws['A3'] = "DEDUCTOR INFORMATION"
        ws['A3'].font = Font(bold=True, size=12, underline="single")

        deductor_fields = [
            ("A4", "TAN", "B4", "Enter 10-character TAN (e.g., DELS00000A)"),
            ("A5", "PAN", "B5", "Enter PAN of deductor"),
            ("A6", "FINANCIAL YEAR", "B6", "e.g., 2024-25"),
            ("A7", "ASSESSMENT YEAR", "B7", "e.g., 2025-26"),
            ("A8", "QUARTER", "B8", "Q1/Q2/Q3/Q4"),
            ("A9", "HAS ANY STATEMENT BEEN FILED EARLIER FOR THIS QUARTER", "B9", "YES/NO"),
            ("A10", "NAME OF THE DEDUCTOR", "B10", "Company/Individual name"),
            ("A11", "TYPE OF DEDUCTOR", "B11", "01-Company, 02-Others"),
            ("A12", "ADDRESS 1", "B12", "Building/Street"),
            ("A13", "ADDRESS 2", "B13", "Area/Locality"),
            ("A14", "ADDRESS 3", "B14", ""),
            ("A15", "ADDRESS 4", "B15", ""),
            ("A16", "ADDRESS 5", "B16", ""),
            ("A17", "STATE", "B17", "State name"),
            ("A18", "PINCODE", "B18", "6-digit PIN"),
            ("A19", "TELEPHONE NO", "B19", "With STD code"),
            ("A20", "TELEPHONE NO (ALTERNATE)", "B20", "Optional"),
            ("A21", "FAX NO", "B21", "Optional"),
            ("A22", "EMAIL", "B22", "Primary email"),
            ("A23", "EMAIL (ALTERNATE)", "B23", "Optional"),
        ]

        # Add deductor fields with placeholders
        for label_cell, label, value_cell, placeholder in deductor_fields:
            ws[label_cell] = label
            ws[label_cell].font = Font(bold=True)
            ws[value_cell] = ""  # Empty for user to fill
            ws[value_cell].fill = PatternFill(start_color="FFFFE0", end_color="FFFFE0", fill_type="solid")
            # Add comment as guidance
            if placeholder:
                ws[value_cell].comment = openpyxl.comments.Comment(placeholder, "TDS System")

        # Responsible Person Section
        ws['A25'] = "PERSON RESPONSIBLE FOR TAX DEDUCTION"
        ws['A25'].font = Font(bold=True, size=12, underline="single")

        responsible_fields = [
            ("A26", "NAME OF THE PERSON RESPONSIBLE", "B26", "Full name"),
            ("A27", "DESIGNATION", "B27", "Job title"),
            ("A28", "ADDRESS 1", "B28", "Building/Street"),
            ("A29", "ADDRESS 2", "B29", "Area/Locality"),
            ("A30", "ADDRESS 3", "B30", ""),
            ("A31", "ADDRESS 4", "B31", ""),
            ("A32", "ADDRESS 5", "B32", ""),
            ("A33", "STATE", "B33", "State name"),
            ("A34", "PINCODE", "B34", "6-digit PIN"),
            ("A35", "TELEPHONE NO", "B35", "With STD code"),
            ("A36", "TELEPHONE NO (ALTERNATE)", "B36", "Optional"),
            ("A37", "FAX NO", "B37", "Optional"),
            ("A38", "EMAIL", "B38", "Primary email"),
            ("A39", "EMAIL (ALTERNATE)", "B39", "Optional"),
            ("A40", "MOBILE NO", "B40", "10-digit mobile"),
            ("A41", "Token No.", "B41", "DSC token number"),
            ("A42", "PAN", "B42", "PAN of responsible person"),
        ]

        # Add responsible person fields
        for label_cell, label, value_cell, placeholder in responsible_fields:
            ws[label_cell] = label
            ws[label_cell].font = Font(bold=True)
            ws[value_cell] = ""  # Empty for user to fill
            ws[value_cell].fill = PatternFill(start_color="E6F3FF", end_color="E6F3FF", fill_type="solid")
            if placeholder:
                ws[value_cell].comment = openpyxl.comments.Comment(placeholder, "TDS System")

        # Add data validations
        # YES/NO dropdown for statement filed
        yes_no_validation = DataValidation(type="list", formula1='"YES,NO"', allow_blank=False)
        yes_no_validation.add(ws['B9'])
        ws.add_data_validation(yes_no_validation)

        # Type of Deductor dropdown
        deductor_type_validation = DataValidation(type="list", formula1='"01-Company,02-Others"', allow_blank=False)
        deductor_type_validation.add(ws['B11'])
        ws.add_data_validation(deductor_type_validation)

        # Quarter dropdown
        quarter_validation = DataValidation(type="list", formula1='"Q1,Q2,Q3,Q4"', allow_blank=False)
        quarter_validation.add(ws['B8'])
        ws.add_data_validation(quarter_validation)

        # Adjust column widths
        ws.column_dimensions['A'].width = 45
        ws.column_dimensions['B'].width = 40

        # Add instructions at the bottom
        ws['A45'] = "INSTRUCTIONS:"
        ws['A45'].font = Font(bold=True, color="FF0000")
        ws['A46'] = "1. Fields highlighted in YELLOW are mandatory for deductor"
        ws['A47'] = "2. Fields highlighted in BLUE are mandatory for responsible person"
        ws['A48'] = "3. Ensure PAN and TAN are in correct format"
        ws['A49'] = "4. All 20 mandatory fields must be filled before processing"

    def _setup_tds_codes_sheet(self, ws):
        """Setup TDS CODES reference sheet"""
        # Headers
        ws['A1'] = "Section Code"
        ws['B1'] = "Nature of Payment"
        ws['C1'] = "TDS Rate %"
        ws['D1'] = "Threshold Limit"

        # Make headers bold
        for col in range(1, 5):
            ws.cell(row=1, column=col).font = Font(bold=True)
            ws.cell(row=1, column=col).fill = PatternFill(start_color="CCE5FF", end_color="CCE5FF", fill_type="solid")

        # Common TDS sections
        tds_sections = [
            ("194A", "Interest other than on securities", "10", "5000"),
            ("194C", "Payment to Contractors", "1-2", "30000"),
            ("194D", "Insurance Commission", "5", "15000"),
            ("194H", "Commission or Brokerage", "5", "15000"),
            ("194I", "Rent - Land & Building", "10", "240000"),
            ("194I", "Rent - Plant & Machinery", "2", "240000"),
            ("194J", "Professional/Technical Services", "10", "30000"),
            ("194Q", "Purchase of Goods", "0.1", "5000000"),
            ("194LA", "Compensation on acquisition of immovable property", "10", "250000"),
            ("194N", "Cash withdrawal", "2", "10000000"),
            ("194O", "E-commerce participants", "1", "500000"),
            ("195", "Payment to Non-residents", "Varies", "Varies"),
            ("94A", "Interest other than on securities", "10", "5000"),
            ("94C", "Payment to Contractors", "1", "30000"),
            ("94I", "Rent", "10", "240000"),
            ("94J", "Professional Services", "10", "30000"),
            ("94Q", "Purchase of Goods", "0.1", "5000000"),
        ]

        # Add data
        for idx, (code, nature, rate, limit) in enumerate(tds_sections, start=2):
            ws.cell(row=idx, column=1).value = code
            ws.cell(row=idx, column=2).value = nature
            ws.cell(row=idx, column=3).value = rate
            ws.cell(row=idx, column=4).value = limit

        # Adjust column widths
        ws.column_dimensions['A'].width = 15
        ws.column_dimensions['B'].width = 50
        ws.column_dimensions['C'].width = 15
        ws.column_dimensions['D'].width = 20

    def _setup_tds_parties_sheet(self, ws):
        """Setup TDS PARTIES sheet - main data entry sheet"""

        # Row 1: Headers
        headers = [
            "Sr.No",
            "Deductee Code (01-Company   02-Others)",
            "Section Under Payment Made",
            "PAN of the Deductee",
            "Name of the deductee",
            "Date of Payment/credit",
            "Amount Paid/Credited Rs.",
            "Paid by Book Entry or otherwise",
            "TDS               Rs.",
            "BSR Code",
            "Challan Serial No",
            "Date on which deposited",
            "TDS Deducted Rates %"
        ]

        # Row 2: Column codes
        codes = [
            "",  # No code for Sr.No
            "(415)",
            "(415A)",
            "(416)",
            "(417)",
            "(418)",
            "(419)",
            "(420)",
            "(421)",
            "(425D)",
            "(425E)",
            "(425F)",
            "(427)"
        ]

        # Write headers (Row 1)
        for col, header in enumerate(headers, 1):
            ws.cell(row=1, column=col).value = header
            ws.cell(row=1, column=col).font = Font(bold=True)
            ws.cell(row=1, column=col).fill = PatternFill(start_color="D3D3D3", end_color="D3D3D3", fill_type="solid")
            ws.cell(row=1, column=col).alignment = Alignment(horizontal='center', wrap_text=True)

        # Write codes (Row 2)
        for col, code in enumerate(codes, 1):
            ws.cell(row=2, column=col).value = code
            ws.cell(row=2, column=col).font = Font(italic=True)
            ws.cell(row=2, column=col).alignment = Alignment(horizontal='center')

        # Add sample/template rows (3-202) - 200 rows for data entry
        for row_num in range(3, 203):
            # Sr.No
            ws.cell(row=row_num, column=1).value = row_num - 2

            # Add formulas for TDS calculation (example)
            # TDS (column 9) could be = Amount (column 7) * Rate (column 13) / 100
            if row_num == 3:  # First data row as example
                ws.cell(row=row_num, column=2).value = "01"  # Deductee Code example
                ws.cell(row=row_num, column=3).value = "94C"  # Section example
                ws.cell(row=row_num, column=4).value = "AAAAA9999A"  # PAN example
                ws.cell(row=row_num, column=5).value = "Sample Company Name"
                ws.cell(row=row_num, column=6).value = datetime.now().date()
                ws.cell(row=row_num, column=7).value = 100000  # Amount example
                ws.cell(row=row_num, column=9).value = "=G3*M3/100"  # TDS formula
                ws.cell(row=row_num, column=13).value = 1  # Rate example

                # Color the example row
                for col in range(1, 14):
                    ws.cell(row=row_num, column=col).fill = PatternFill(start_color="F0F8FF", end_color="F0F8FF", fill_type="solid")
            else:
                # Empty rows for user entry, but with formulas
                ws.cell(row=row_num, column=9).value = f"=IF(G{row_num}<>\"\",G{row_num}*M{row_num}/100,\"\")"

        # Add data validation for Section codes
        section_validation = DataValidation(
            type="list",
            formula1='"94A,94C,94D,94H,94I,94J,94Q,194A,194C,194D,194H,194I,194J,194Q"',
            allow_blank=True
        )
        section_validation.add(f'C3:C202')
        ws.add_data_validation(section_validation)

        # Add data validation for Deductee Code
        deductee_validation = DataValidation(type="list", formula1='"01,02"', allow_blank=True)
        deductee_validation.add(f'B3:B202')
        ws.add_data_validation(deductee_validation)

        # Set column widths
        column_widths = {
            'A': 8,   # Sr.No
            'B': 15,  # Deductee Code
            'C': 20,  # Section
            'D': 15,  # PAN
            'E': 35,  # Name
            'F': 15,  # Date of Payment
            'G': 20,  # Amount
            'H': 15,  # Book Entry
            'I': 15,  # TDS
            'J': 12,  # BSR Code
            'K': 15,  # Challan No
            'L': 15,  # Date Deposited
            'M': 10,  # Rate
        }

        for col_letter, width in column_widths.items():
            ws.column_dimensions[col_letter].width = width

        # Format date columns
        for row in range(3, 203):
            ws.cell(row=row, column=6).number_format = 'DD/MM/YYYY'
            ws.cell(row=row, column=12).number_format = 'DD/MM/YYYY'

        # Add instructions row
        ws.cell(row=204, column=1).value = "Note: BSR Code, Challan No, and Date Deposited will be auto-filled from PDF extraction"
        ws.merge_cells('A204:M204')
        ws.cell(row=204, column=1).font = Font(italic=True, color="FF0000")

    def _setup_challan_details_sheet(self, ws):
        """Setup Challan Details sheet (system fills this from PDFs)"""
        # Headers
        headers = [
            "Sr.No",
            "Nature of Payment",
            "Tax",
            "Surcharge",
            "Cess",
            "Interest",
            "Penalty",
            "Total",
            "Mode of Payment",
            "BSR Code",
            "Tender Date",
            "Challan No",
            "Book Entry"
        ]

        # Row 1: Title
        ws['A1'] = "CHALLAN DETAILS (AUTO-FILLED FROM PDFs)"
        ws['A1'].font = Font(bold=True, size=12, color="FF0000")
        ws.merge_cells('A1:M1')

        # Row 2: Headers
        for col, header in enumerate(headers, 1):
            ws.cell(row=2, column=col).value = header
            ws.cell(row=2, column=col).font = Font(bold=True)
            ws.cell(row=2, column=col).fill = PatternFill(start_color="FFE5E5", end_color="FFE5E5", fill_type="solid")

        # Add note
        ws['A3'] = "This sheet will be automatically filled when you upload PDF files"
        ws.merge_cells('A3:M3')
        ws['A3'].font = Font(italic=True)

        # Set column widths
        for col in range(1, 14):
            ws.column_dimensions[get_column_letter(col)].width = 15

    def _setup_tds_rates_sheet(self, ws):
        """Setup TDS RATES reference sheet"""
        # Title
        ws['A1'] = "TDS RATES - QUICK REFERENCE"
        ws['A1'].font = Font(bold=True, size=14)
        ws.merge_cells('A1:D1')

        # Headers
        ws['A3'] = "Category"
        ws['B3'] = "Section"
        ws['C3'] = "Threshold (Rs.)"
        ws['D3'] = "Rate %"

        for col in range(1, 5):
            ws.cell(row=3, column=col).font = Font(bold=True)
            ws.cell(row=3, column=col).fill = PatternFill(start_color="CCE5FF", end_color="CCE5FF", fill_type="solid")

        # Common rates
        rates_data = [
            ("Salary", "192", "Basic Exemption", "Slab Rates"),
            ("Interest (Bank)", "194A", "40000 (General) / 50000 (Senior)", "10"),
            ("Contractors - Individual", "194C", "30000 (Single) / 100000 (Total)", "1"),
            ("Contractors - Company", "194C", "30000 (Single) / 100000 (Total)", "2"),
            ("Insurance Commission", "194D", "15000", "5 (Ind) / 10 (Co)"),
            ("Commission/Brokerage", "194H", "15000", "5"),
            ("Rent - Land/Building", "194I", "240000", "10"),
            ("Rent - Plant/Machinery", "194I", "240000", "2"),
            ("Professional Services", "194J", "30000", "10 (General) / 2 (Specified)"),
            ("Purchase of Goods", "194Q", "5000000", "0.1"),
        ]

        for idx, (category, section, threshold, rate) in enumerate(rates_data, start=4):
            ws.cell(row=idx, column=1).value = category
            ws.cell(row=idx, column=2).value = section
            ws.cell(row=idx, column=3).value = threshold
            ws.cell(row=idx, column=4).value = rate

        # Adjust column widths
        ws.column_dimensions['A'].width = 30
        ws.column_dimensions['B'].width = 15
        ws.column_dimensions['C'].width = 30
        ws.column_dimensions['D'].width = 20

        # Add notes
        ws['A15'] = "IMPORTANT NOTES:"
        ws['A15'].font = Font(bold=True, color="FF0000")
        ws['A16'] = "1. These are basic rates. Special rates may apply in certain cases."
        ws['A17'] = "2. No TDS if PAN is provided and payment is below threshold."
        ws['A18'] = "3. 20% TDS if PAN is not provided (Section 206AA)."
        ws['A19'] = "4. Check latest Finance Act for updated rates."

    def save_template(self, filename="TDS_Masters_Template.xlsx"):
        """Save the template to a file"""
        if self.wb:
            self.wb.save(filename)
            return filename
        return None

def generate_tds_masters_template():
    """
    Main function to generate and save TDS_Masters template
    Returns the filename of created template
    """
    print("📝 Generating TDS_Masters Template...")

    generator = TDSMastersTemplateGenerator()
    generator.create_template()

    filename = "TDS_Masters_Template.xlsx"
    generator.save_template(filename)

    print(f"✅ Template created successfully: {filename}")
    print("\n📋 Template contains:")
    print("   1. INPUT-DEDUCTOR DETAILS - Fill company & responsible person info (20 mandatory fields)")
    print("   2. TDS CODES - Reference sheet with section codes")
    print("   3. TDS PARTIES - Main data entry (200 rows, fill as needed)")
    print("   4. Challan Details - Leave blank (system fills from PDFs)")
    print("   5. TDS RATES - Quick reference for TDS rates")
    print("\n💡 Instructions:")
    print("   - Yellow cells: Mandatory deductor fields")
    print("   - Blue cells: Mandatory responsible person fields")
    print("   - Row 3 in TDS PARTIES has sample data")
    print("   - Formulas auto-calculate TDS based on amount and rate")

    return filename

# Widget function for download button (if using widgets)
def create_template_download_widget():
    """Create a widget interface for template download"""
    import ipywidgets as widgets
    from IPython.display import display

    def download_template(b):
        filename = generate_tds_masters_template()
        print(f"\n📥 Ready to download: {filename}")
        # In Colab, use files.download()
        from google.colab import files
        files.download(filename)

    button = widgets.Button(
        description='📥 Download TDS_Masters Template',
        button_style='primary',
        tooltip='Download blank template for filling party details',
        layout=widgets.Layout(width='300px', height='50px')
    )

    button.on_click(download_template)

    info = widgets.HTML(
        value="""
        <div style='padding: 10px; background: #f0f0f0; border-radius: 5px; margin: 10px 0;'>
            <h3>📋 TDS_Masters Template</h3>
            <p>Download this template if you need a blank form to fill in:</p>
            <ul>
                <li>Company/Deductor details (20 mandatory fields)</li>
                <li>Party-wise TDS details (up to 200 entries)</li>
                <li>Reference sheets for TDS codes and rates</li>
            </ul>
            <p><b>After filling, upload this file along with PDF challans for processing.</b></p>
        </div>
        """
    )

    return widgets.VBox([info, button])

print("✅ Cell 8: TDS_Masters Template Generator loaded successfully!")
print("   Functions available:")
print("   - generate_tds_masters_template() : Generate and download template")
print("   - create_template_download_widget() : Create download widget")
print("\n📌 This template is the starting point for users who don't have TDS_Masters.xlsx")

✅ Cell 8: TDS_Masters Template Generator loaded successfully!
   Functions available:
   - generate_tds_masters_template() : Generate and download template
   - create_template_download_widget() : Create download widget

📌 This template is the starting point for users who don't have TDS_Masters.xlsx


In [5]:
# Cell 5: Government Template Builder and Enhanced Output Generator
# This cell creates the government template internally and fills it with data
# No need to upload TDS_Template.xlsx anymore!
# FIXED: Dynamic row handling - only fills actual data rows, adds TOTAL after

import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.utils import get_column_letter
from decimal import Decimal, ROUND_HALF_UP
import pandas as pd
from datetime import datetime
import math

class GovernmentTemplateBuilder:
    """
    Build TDS government return format internally without requiring template upload
    Generates the exact structure of TDS_Template.xlsx with dynamic row handling
    """

    def __init__(self):
        self.wb = None

    def create_template(self):
        """Create blank government format template with all three sheets"""
        self.wb = Workbook()

        # Create DEDUCTOR DETAILS sheet
        ws_deductor = self.wb.active
        ws_deductor.title = "DEDUCTOR DETAILS"
        self._setup_deductor_sheet(ws_deductor)

        # Create CHALLAN DETAILS sheet
        ws_challan = self.wb.create_sheet("CHALLAN DETAILS")
        self._setup_challan_sheet(ws_challan)

        # Create DEDUCTEE BREAK-UP sheet
        ws_deductee = self.wb.create_sheet("DEDUCTEE BREAK-UP")
        self._setup_deductee_sheet(ws_deductee)

        return self.wb

    def _setup_deductor_sheet(self, ws):
        """Setup DEDUCTOR DETAILS sheet structure"""
        # Headers for deductor details - matching government template format
        deductor_fields = [
            ("A1", "TAN"),
            ("A2", "PAN"),
            ("A3", "FINANCIAL YEAR"),
            ("A4", "ASSESSMENT YEAR"),
            ("A5", "QUARTER"),
            ("A6", "HAS ANY STATEMENT BEEN FILED EARLIER FOR THIS QUARTER"),
            ("A7", "NAME OF THE DEDUCTOR"),
            ("A8", "TYPE OF DEDUCTOR"),
            ("A9", "ADDRESS 1"),
            ("A10", "ADDRESS 2"),
            ("A11", "ADDRESS 3"),
            ("A12", "ADDRESS 4"),
            ("A13", "ADDRESS 5"),
            ("A14", "STATE"),
            ("A15", "PINCODE"),
            ("A16", "TELEPHONE NO"),
            ("A17", "TELEPHONE NO (ALTERNATE)"),
            ("A18", "FAX NO"),
            ("A19", "EMAIL"),
            ("A20", "EMAIL (ALTERNATE)"),
            # Responsible person details
            ("A22", "NAME OF THE PERSON RESPONSIBLE FOR DEDUCTION OF TAX"),
            ("A23", "DESIGNATION"),
            ("A24", "ADDRESS 1"),
            ("A25", "ADDRESS 2"),
            ("A26", "ADDRESS 3"),
            ("A27", "ADDRESS 4"),
            ("A28", "ADDRESS 5"),
            ("A29", "STATE"),
            ("A30", "PINCODE"),
            ("A31", "TELEPHONE NO"),
            ("A32", "TELEPHONE NO (ALTERNATE)"),
            ("A33", "FAX NO"),
            ("A34", "EMAIL"),
            ("A35", "EMAIL (ALTERNATE)"),
            ("A36", "MOBILE NO"),
            ("A37", "Token No."),
            ("A38", "PAN"),
        ]

        # Add field labels
        for cell, label in deductor_fields:
            ws[cell] = label
            ws[cell].font = Font(bold=True)

        # Adjust column width
        ws.column_dimensions['A'].width = 50
        ws.column_dimensions['B'].width = 40

    def _setup_challan_sheet(self, ws):
        """Setup CHALLAN DETAILS sheet structure"""
        # Row 1: Leave empty (as per original template)
        # Row 2: Column codes
        codes_row2 = [
            "(401)", "(402)", "(403)", "(404)", "(405)",
            "(406)", "(407)", "(408)", "(409)", "(410)",
            "(411)", "(412)", "(413)"
        ]

        # Row 3: Column headers
        headers_row3 = [
            "SR NO", "SECTION CODE", "TDS Rs.", "SURCHARGE Rs.",
            "EDUCATION CESS Rs.", "INTEREST Rs.", "OTHERS Rs.",
            "TOTAL TAX DEPOSITED", "CHEQUE/DD NO", "BSR CODE",
            "DATE ON WHICH TAX DEPOSITED", "CHALLAN SERIAL NO",
            "WHETHER TDS DEPOSITED BY BOOK ENTRY"
        ]

        # Write codes and headers (Row 1 stays empty)
        for col, code in enumerate(codes_row2, 1):
            ws.cell(row=2, column=col, value=code)
            ws.cell(row=2, column=col).alignment = Alignment(horizontal='center')

        for col, header in enumerate(headers_row3, 1):
            ws.cell(row=3, column=col, value=header)
            ws.cell(row=3, column=col).font = Font(bold=True)
            ws.cell(row=3, column=col).alignment = Alignment(horizontal='center', wrap_text=True)

        # Set column widths
        column_widths = [8, 15, 12, 12, 12, 12, 12, 12, 15, 12, 20, 15, 12]
        for col, width in enumerate(column_widths, 1):
            ws.column_dimensions[get_column_letter(col)].width = width

    def _setup_deductee_sheet(self, ws):
        """Setup DEDUCTEE BREAK-UP sheet structure"""
        # Row 1: Leave empty (as per original template)
        # Row 2: Column codes (414 to 428)
        codes_row2 = [
            "(414)", "(415)", "(415A)", "(416)", "(417)", "(418)", "(419)",
            "(420)", "(421)", "(422)", "(423)", "(424)", "(425)", "(425A)",
            "(425B)", "(425C)", "(425D)", "(425E)", "(425F)", "(426)",
            "(427)", "(428)"
        ]

        # Row 3: Column headers
        headers_row3 = [
            "Sr.No", "Deductee Code (01-Company 02-Others)",
            "Section Under Payment Made", "PAN of the Deductee",
            "Name of the deductee", "Date of Payment/credit",
            "Amount Paid/Credited Rs.", "Paid by Book Entry or otherwise",
            "TDS Rs.", "Surcharge Rs.", "Educational Cess Rs.",
            "Total tax deducted (421+422+423)", "Total tax deposited Rs.",
            "Interest", "Others", "Total (425+Interest+Others)",
            "BSR Code", "Challan Serial No", "Date on which deposited",
            "Date of deduction", "Rate at which deducted",
            "Reason for non-deduction/lower deduction"
        ]

        # Write codes and headers (Row 1 stays empty)
        for col, code in enumerate(codes_row2, 1):
            ws.cell(row=2, column=col, value=code)
            ws.cell(row=2, column=col).alignment = Alignment(horizontal='center')

        for col, header in enumerate(headers_row3, 1):
            ws.cell(row=3, column=col, value=header)
            ws.cell(row=3, column=col).font = Font(bold=True)
            ws.cell(row=3, column=col).alignment = Alignment(horizontal='center', wrap_text=True)

        # Set column widths
        column_widths = [8, 12, 12, 15, 30, 15, 15, 12, 12, 12, 12, 12, 12, 10, 10, 12, 12, 15, 15, 15, 10, 30]
        for col, width in enumerate(column_widths, 1):
            ws.column_dimensions[get_column_letter(col)].width = width

    def fill_template(self, masters_data, challan_data_list):
        """
        Fill the government template with data from Masters and Challans

        Args:
            masters_data: Dictionary containing TDS Masters data
            challan_data_list: List of challan dictionaries from PDF extraction
        """
        if not self.wb:
            self.create_template()

        # Fill DEDUCTOR DETAILS
        self._fill_deductor_details(masters_data)

        # Fill CHALLAN DETAILS
        self._fill_challan_details(challan_data_list)

        # Fill DEDUCTEE BREAK-UP
        self._fill_deductee_breakup(masters_data, challan_data_list)

        return self.wb

    def _fill_deductor_details(self, masters_data):
        """Map and fill DEDUCTOR DETAILS from INPUT-DEDUCTOR DETAILS"""
        ws = self.wb["DEDUCTOR DETAILS"]

        # Check if INPUT-DEDUCTOR DETAILS exists
        if 'INPUT-DEDUCTOR DETAILS' not in masters_data:
            print("⚠️ Warning: INPUT-DEDUCTOR DETAILS sheet not found in Masters")
            return

        deductor_df = masters_data['INPUT-DEDUCTOR DETAILS']

        # Map fields from INPUT-DEDUCTOR DETAILS to DEDUCTOR DETAILS
        # Assuming data is in column B of the input sheet
        field_mapping = {
            'B1': 'B1',   # TAN
            'B2': 'B2',   # PAN
            'B3': 'B3',   # FINANCIAL YEAR
            'B4': 'B4',   # ASSESSMENT YEAR
            'B5': 'B5',   # QUARTER
            'B6': 'B6',   # HAS STATEMENT BEEN FILED
            'B7': 'B7',   # NAME OF DEDUCTOR
            'B8': 'B8',   # TYPE OF DEDUCTOR
            'B9': 'B9',   # ADDRESS 1
            'B10': 'B10', # ADDRESS 2
            'B11': 'B11', # ADDRESS 3
            'B12': 'B12', # ADDRESS 4
            'B13': 'B13', # ADDRESS 5
            'B14': 'B14', # STATE
            'B15': 'B15', # PINCODE
            'B16': 'B16', # TELEPHONE NO
            'B17': 'B17', # TELEPHONE NO (ALTERNATE)
            'B18': 'B18', # FAX NO
            'B19': 'B19', # EMAIL
            'B20': 'B20', # EMAIL (ALTERNATE)
            # Responsible person details
            'B22': 'B22', # NAME OF PERSON RESPONSIBLE
            'B23': 'B23', # DESIGNATION
            'B24': 'B24', # ADDRESS 1
            'B25': 'B25', # ADDRESS 2
            'B26': 'B26', # ADDRESS 3
            'B27': 'B27', # ADDRESS 4
            'B28': 'B28', # ADDRESS 5
            'B29': 'B29', # STATE
            'B30': 'B30', # PINCODE
            'B31': 'B31', # TELEPHONE NO
            'B32': 'B32', # TELEPHONE NO (ALTERNATE)
            'B33': 'B33', # FAX NO
            'B34': 'B34', # EMAIL
            'B35': 'B35', # EMAIL (ALTERNATE)
            'B36': 'B36', # MOBILE NO
            'B37': 'B37', # Token No.
            'B38': 'B38', # PAN
        }

        # If deductor_df is a DataFrame, convert to dict or access directly
        if isinstance(deductor_df, pd.DataFrame):
            # Assuming the data is in the first two columns (field name, value)
            for idx, row in deductor_df.iterrows():
                try:
                    # Map row number to cell position
                    target_cell = f'B{idx + 1}'
                    if idx < len(deductor_df):
                        value = row.iloc[1] if len(row) > 1 else ''
                        if pd.notna(value):
                            ws[target_cell] = str(value)
                except:
                    continue

        print("✅ DEDUCTOR DETAILS filled from INPUT-DEDUCTOR DETAILS")

    def _fill_challan_details(self, challan_data_list):
        """Fill CHALLAN DETAILS sheet with PDF extracted data"""
        ws = self.wb["CHALLAN DETAILS"]

        # Data starts from row 4 (row 1 empty, row 2 codes, row 3 headers)
        start_row = 4

        # Only process actual challans (already deduplicated)
        actual_challan_count = len(challan_data_list)

        for idx, challan in enumerate(challan_data_list, start=1):
            row = start_row + idx - 1

            # Round amounts
            tax_amt = math.ceil(float(challan.get('tax_amount', 0))) if challan.get('tax_amount', '') else 0
            surcharge = math.ceil(float(challan.get('surcharge', 0))) if challan.get('surcharge', '') else 0
            cess = math.ceil(float(challan.get('cess', 0))) if challan.get('cess', '') else 0
            interest = math.ceil(float(challan.get('interest', 0))) if challan.get('interest', '') else 0
            penalty = math.ceil(float(challan.get('penalty', 0))) if challan.get('penalty', '') else 0

            # Fill columns
            ws.cell(row=row, column=1).value = idx  # Sr.No (401)
            ws.cell(row=row, column=2).value = challan.get('nature_of_payment', '')  # Section Code (402)
            ws.cell(row=row, column=3).value = tax_amt  # TDS (403)
            ws.cell(row=row, column=4).value = surcharge  # Surcharge (404)
            ws.cell(row=row, column=5).value = cess  # Education Cess (405)
            ws.cell(row=row, column=6).value = interest  # Interest (406)
            ws.cell(row=row, column=7).value = penalty  # Others (407)
            ws.cell(row=row, column=8).value = f'=SUM(C{row}:G{row})'  # Total (408)
            ws.cell(row=row, column=9).value = challan.get('mode_of_payment', '')  # Cheque/DD (409)
            ws.cell(row=row, column=10).value = challan.get('bsr_code', '')  # BSR Code (410)

            # Date formatting
            date_str = challan.get('tender_date', '')
            if date_str:
                try:
                    date_obj = datetime.strptime(date_str, '%d/%m/%Y')
                    ws.cell(row=row, column=11).value = date_obj
                    ws.cell(row=row, column=11).number_format = 'DD/MM/YYYY'
                except:
                    ws.cell(row=row, column=11).value = date_str

            ws.cell(row=row, column=12).value = challan.get('challan_no', '')  # Challan Serial No (412)
            ws.cell(row=row, column=13).value = 'NO'  # Book Entry (413)

        # Add TOTAL row immediately after the last data row
        total_row = start_row + actual_challan_count
        ws.cell(row=total_row, column=2).value = "TOTAL"
        ws.cell(row=total_row, column=2).font = Font(bold=True)

        # Add sum formulas for total row
        last_data_row = total_row - 1
        for col in [3, 4, 5, 6, 7, 8]:  # Columns C to H
            ws.cell(row=total_row, column=col).value = f'=SUM({get_column_letter(col)}{start_row}:{get_column_letter(col)}{last_data_row})'
            ws.cell(row=total_row, column=col).font = Font(bold=True)

        print(f"✅ CHALLAN DETAILS filled with {actual_challan_count} challans, TOTAL row at {total_row}")

    def _fill_deductee_breakup(self, masters_data, challan_data_list):
        """Fill DEDUCTEE BREAK-UP sheet with party data and challan matching"""
        ws = self.wb["DEDUCTEE BREAK-UP"]

        # Get TDS PARTIES data
        if 'tds_parties' not in masters_data:
            print("⚠️ Warning: TDS PARTIES data not found")
            return

        tds_parties = masters_data['tds_parties']
        code_to_column_name = masters_data.get('code_to_column_name', {})

        # Count actual valid parties (non-empty rows)
        valid_parties = []
        payment_col = code_to_column_name.get('(415A)', '')
        name_col = code_to_column_name.get('(417)', '')

        for _, party in tds_parties.iterrows():
            # Check if this is a valid party row (has payment type or name)
            payment_type = party.get(payment_col, '') if payment_col else ''
            name = party.get(name_col, '') if name_col else ''

            # Skip empty rows
            if (payment_type and str(payment_type).strip() not in ['', 'nan', 'None']) or \
               (name and str(name).strip() not in ['', 'nan', 'None']):
                valid_parties.append(party)

        print(f"   Found {len(valid_parties)} valid parties out of {len(tds_parties)} total rows")

        # Create challan lookup by nature of payment
        challan_lookup = {}
        for challan in challan_data_list:
            nop = challan.get('nature_of_payment', '').replace(' ', '')
            challan_lookup[nop] = challan

        # Data starts from row 4 (row 1 empty, row 2 codes, row 3 headers)
        start_row = 4
        row_idx = start_row

        # Process only valid parties
        for party in valid_parties:
            # Get payment type for challan matching
            payment_type = party.get(payment_col, '') if payment_col else ''
            payment_type_clean = str(payment_type).replace(' ', '')
            challan = challan_lookup.get(payment_type_clean, {})

            # Fill columns
            ws.cell(row=row_idx, column=1).value = row_idx - start_row + 1  # Sr.No (414)

            # Deductee Code (415)
            deductee_code = party.get(code_to_column_name.get('(415)', ''), '') if code_to_column_name.get('(415)') else ''
            if not deductee_code and code_to_column_name.get('(416)'):
                # Determine from PAN if not provided
                pan = str(party.get(code_to_column_name.get('(416)', ''), ''))
                if len(pan) >= 4:
                    deductee_code = '02' if pan[3].upper() == 'P' else '01'
            ws.cell(row=row_idx, column=2).value = deductee_code

            # Section Under Payment (415A)
            ws.cell(row=row_idx, column=3).value = payment_type

            # PAN (416)
            ws.cell(row=row_idx, column=4).value = party.get(code_to_column_name.get('(416)', ''), '')

            # Name (417)
            ws.cell(row=row_idx, column=5).value = party.get(code_to_column_name.get('(417)', ''), '')

            # Date of Payment (418)
            date_payment = party.get(code_to_column_name.get('(418)', ''), '')
            if pd.notna(date_payment):
                ws.cell(row=row_idx, column=6).value = date_payment
                ws.cell(row=row_idx, column=6).number_format = 'DD/MM/YYYY'

            # Amount Paid (419)
            amount_paid = party.get(code_to_column_name.get('(419)', ''), 0)
            if pd.notna(amount_paid):
                ws.cell(row=row_idx, column=7).value = int(float(str(amount_paid)))

            # TDS Amount (421)
            tds_amount = party.get(code_to_column_name.get('(421)', ''), 0)
            if pd.notna(tds_amount):
                ws.cell(row=row_idx, column=9).value = int(float(str(tds_amount)))

            # Surcharge and Cess (422, 423)
            ws.cell(row=row_idx, column=10).value = 0
            ws.cell(row=row_idx, column=11).value = 0

            # Total tax deducted formula (424)
            ws.cell(row=row_idx, column=12).value = f'=SUM(I{row_idx}:K{row_idx})'

            # Total tax deposited (425)
            ws.cell(row=row_idx, column=13).value = f'=L{row_idx}'

            # Interest and Others (425A, 425B)
            ws.cell(row=row_idx, column=14).value = 0
            ws.cell(row=row_idx, column=15).value = 0

            # Total formula (425C)
            ws.cell(row=row_idx, column=16).value = f'=SUM(M{row_idx}:O{row_idx})'

            # BSR Code, Challan No, Date from challan matching (425D, 425E, 425F)
            ws.cell(row=row_idx, column=17).value = challan.get('bsr_code', '')
            ws.cell(row=row_idx, column=18).value = challan.get('challan_no', '')

            date_str = challan.get('tender_date', '')
            if date_str:
                try:
                    date_obj = datetime.strptime(date_str, '%d/%m/%Y')
                    ws.cell(row=row_idx, column=19).value = date_obj
                    ws.cell(row=row_idx, column=19).number_format = 'DD/MM/YYYY'
                except:
                    ws.cell(row=row_idx, column=19).value = date_str

            # Date of deduction (426)
            if pd.notna(date_payment):
                ws.cell(row=row_idx, column=20).value = date_payment
                ws.cell(row=row_idx, column=20).number_format = 'DD/MM/YYYY'

            # Rate at which deducted (427)
            rate = party.get(code_to_column_name.get('(427)', ''), 0)
            if pd.notna(rate):
                try:
                    rate_val = float(str(rate))
                    if rate_val < 1:  # Convert decimal to percentage
                        rate_val = rate_val * 100
                    ws.cell(row=row_idx, column=21).value = f'{rate_val:.2f}%'
                except:
                    ws.cell(row=row_idx, column=21).value = str(rate)

            # Reason for non-deduction (428)
            ws.cell(row=row_idx, column=22).value = 'N.A'

            row_idx += 1

        # Add TOTAL row immediately after the last data row
        total_row = row_idx
        last_data_row = row_idx - 1
        ws.cell(row=total_row, column=1).value = "TOTAL"
        ws.cell(row=total_row, column=1).font = Font(bold=True)

        # Add sum formulas for numeric columns
        sum_columns = [7, 9, 10, 11, 12, 13, 14, 15, 16]  # Columns G, I, J, K, L, M, N, O, P
        for col in sum_columns:
            if last_data_row >= start_row:  # Only add formula if there's data
                ws.cell(row=total_row, column=col).value = f'=SUM({get_column_letter(col)}{start_row}:{get_column_letter(col)}{last_data_row})'
                ws.cell(row=total_row, column=col).font = Font(bold=True)
            else:
                ws.cell(row=total_row, column=col).value = 0
                ws.cell(row=total_row, column=col).font = Font(bold=True)

        print(f"✅ DEDUCTEE BREAK-UP filled with {len(valid_parties)} parties, TOTAL row at {total_row}")

def generate_output_file_enhanced(masters_data, challan_data_list, output_path=None):
    """
    Enhanced output file generator using internal template
    No need to upload government template anymore!

    Args:
        masters_data: Dictionary from read_tds_masters()
        challan_data_list: List from extract_all_challans()
        output_path: Where to save the file

    Returns:
        Path to generated file
    """
    # Generate output filename if not provided
    if not output_path:
        # Try to get from payment dates
        code_to_column_name = masters_data.get('code_to_column_name', {})
        date_col = code_to_column_name.get('(418)')

        if date_col and 'tds_parties' in masters_data:
            dates = masters_data['tds_parties'][date_col].dropna()
            if not dates.empty:
                first_date = pd.to_datetime(dates.iloc[0])
                output_path = f"TDS_{first_date.strftime('%B')}_{first_date.strftime('%Y')}.xlsx"
            else:
                output_path = f"TDS_{datetime.now().strftime('%B')}_{datetime.now().strftime('%Y')}.xlsx"
        else:
            output_path = f"TDS_{datetime.now().strftime('%B')}_{datetime.now().strftime('%Y')}.xlsx"

    # Create and fill template
    builder = GovernmentTemplateBuilder()
    filled_wb = builder.fill_template(masters_data, challan_data_list)

    # Save the file
    filled_wb.save(output_path)
    print(f"\n✅ Generated TDS Return file: {output_path}")
    print("   No template upload needed - generated internally!")
    print("   Only filled actual data rows - no empty rows!")
    print("   TOTAL rows placed correctly after data!")

    return output_path

# Test function to verify the template structure
def test_template_generation():
    """Test function to create a sample government template"""
    print("🧪 Testing Government Template Generation...")

    builder = GovernmentTemplateBuilder()
    wb = builder.create_template()

    # Save test template
    test_path = "TEST_Government_Template.xlsx"
    wb.save(test_path)

    print(f"✅ Test template created: {test_path}")
    print("   Sheets created:")
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        print(f"   - {sheet}: {ws.max_row} rows, {ws.max_column} columns")

    return test_path

print("✅ Cell 5: Government Template Builder loaded successfully!")
print("   Functions available:")
print("   - GovernmentTemplateBuilder() : Main template builder class")
print("   - generate_output_file_enhanced() : Generate output without template upload")
print("   - test_template_generation() : Test template creation")
print("\n📌 Key improvements:")
print("   ✅ No need to upload TDS_Template.xlsx")
print("   ✅ Row 1 left empty as per original template")
print("   ✅ Only fills actual data rows (skips empty rows from Masters)")
print("   ✅ TOTAL row placed immediately after last data row")
print("   ✅ Handles 10, 20, or any number of parties dynamically")

✅ Cell 5: Government Template Builder loaded successfully!
   Functions available:
   - GovernmentTemplateBuilder() : Main template builder class
   - generate_output_file_enhanced() : Generate output without template upload
   - test_template_generation() : Test template creation

📌 Key improvements:
   ✅ No need to upload TDS_Template.xlsx
   ✅ Row 1 left empty as per original template
   ✅ Only fills actual data rows (skips empty rows from Masters)
   ✅ TOTAL row placed immediately after last data row
   ✅ Handles 10, 20, or any number of parties dynamically


In [6]:
# Cell 9: Enhanced Main Processing Pipeline
# This integrates validation, internal template generation, and reporting
# NO TEMPLATE UPLOAD NEEDED - Government template generated internally
# FIXED: Removed duplicate _fill_ functions, now uses GovernmentTemplateBuilder from Cell 5

import os
import math
import pandas as pd
import numpy as np
from datetime import datetime
from decimal import Decimal, ROUND_HALF_UP
import openpyxl
from openpyxl import load_workbook, Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill

def get_output_filename_from_masters(tds_masters_data):
    """
    Extract month and year from the first payment date in TDS Masters
    to generate output filename as TDS_Month_Year.xlsx
    """
    try:
        code_to_column_name = tds_masters_data.get('code_to_column_name', {})
        date_col = code_to_column_name.get('(418)')
        if date_col and date_col in tds_masters_data['tds_parties'].columns:
            dates = tds_masters_data['tds_parties'][date_col].dropna()
            if not dates.empty:
                first_date = pd.to_datetime(dates.iloc[0])
                month_name = first_date.strftime('%B')
                year = first_date.strftime('%Y')
                return f"TDS_{month_name}_{year}.xlsx"
        current_date = datetime.now()
        return f"TDS_{current_date.strftime('%B')}_{current_date.strftime('%Y')}.xlsx"
    except:
        current_date = datetime.now()
        return f"TDS_{current_date.strftime('%B')}_{current_date.strftime('%Y')}.xlsx"

def fill_government_template(wb, tds_masters_data, challan_data_list):
    """
    Fill the internally generated government template with data
    Now uses GovernmentTemplateBuilder from Cell 5 for the actual filling logic

    Args:
        wb: Workbook object (not used - kept for backward compatibility)
        tds_masters_data: Dictionary containing TDS Masters data
        challan_data_list: List of challan dictionaries from PDF extraction

    Returns:
        Filled workbook object
    """
    print("\n📝 Filling Government Template...")

    # Import the GovernmentTemplateBuilder from Cell 5
    try:
        from GovernmentTemplateBuilder import GovernmentTemplateBuilder
        print("   Using GovernmentTemplateBuilder from Cell 5...")

        # Create builder and fill template
        builder = GovernmentTemplateBuilder()
        filled_wb = builder.fill_template(tds_masters_data, challan_data_list)

        print("✅ Government template filled successfully")
        return filled_wb

    except ImportError:
        print("⚠️ GovernmentTemplateBuilder not found. Please run Cell 5 first.")
        raise ImportError("GovernmentTemplateBuilder class not found. Ensure Cell 5 is executed first.")

def process_tds_returns_enhanced(pdf_folder_path, masters_file_path, output_folder=None):
    """
    Enhanced main processing function - NO TEMPLATE UPLOAD NEEDED
    Integrates validation, internal template generation, and reconciliation report

    Args:
        pdf_folder_path: Path to folder containing PDF challans
        masters_file_path: Path to TDS_Masters.xlsx file
        output_folder: Optional output folder path

    Returns:
        Dictionary with all output paths and validation results
    """
    print("\n" + "="*60)
    print("🚀 STARTING ENHANCED TDS RETURN PROCESSING")
    print("="*60)

    # Set output folder
    if not output_folder:
        output_folder = os.path.dirname(masters_file_path)

    # Step 1: Extract PDFs (existing function)
    print("\n📄 STEP 1: Extracting Challan Data from PDFs...")
    print("-" * 50)
    challan_data_list = extract_all_challans(pdf_folder_path)

    if not challan_data_list:
        print("❌ No valid challans extracted. Aborting.")
        return None

    print(f"✅ Successfully extracted {len(challan_data_list)} unique challans")

    # Step 2: Read Masters (existing function)
    print("\n📊 STEP 2: Reading TDS Masters File...")
    print("-" * 50)
    tds_masters_data = read_tds_masters(masters_file_path)

    if not tds_masters_data:
        print("❌ Failed to read TDS Masters. Aborting.")
        return None

    print("✅ Successfully read TDS Masters file")

    # Step 3: Validate mandatory fields
    print("\n🔍 STEP 3: Validating Mandatory Fields...")
    print("-" * 50)

    try:
        validator = ValidationEngine(
            masters_file_path=masters_file_path,
            masters_data=tds_masters_data,
            challan_data_list=challan_data_list
        )
        validation_results = validator.validate_all()

        # Check validation status but don't block (warn-only)
        validation_status = validation_results['summary']['validation_status']
        if validation_status == 'CRITICAL':
            print("⚠️ Critical validation issues found - proceeding anyway (warn-only mode)")
        elif validation_status == 'WARNING':
            print("⚠️ Minor validation issues found - proceeding")
        else:
            print("✅ All validations passed!")
    except Exception as e:
        print(f"⚠️ Validation engine not available: {str(e)}")
        print("   Proceeding without validation...")
        validation_results = None

    # Step 4: Update Masters with challans (existing function)
    print("\n📝 STEP 4: Updating TDS Masters with Challan Information...")
    print("-" * 50)
    updated_masters_data = update_tds_masters_with_challans(tds_masters_data, challan_data_list)

    if not updated_masters_data:
        print("❌ Failed to update TDS Masters. Aborting.")
        return None

    print("✅ Successfully updated TDS Masters")

    # Step 5: Generate government template internally
    print("\n📋 STEP 5: Generating Government Template Internally...")
    print("-" * 50)

    try:
        # Import GovernmentTemplateBuilder from Cell 5
        from GovernmentTemplateBuilder import GovernmentTemplateBuilder

        # Create template builder
        builder = GovernmentTemplateBuilder()

        # Create blank template
        govt_template_wb = builder.create_template()

        # Fill the template with data
        filled_template_wb = builder.fill_template(updated_masters_data, challan_data_list)
        print("✅ Using GovernmentTemplateBuilder from Cell 5")

    except ImportError:
        print("⚠️ GovernmentTemplateBuilder not found, creating template manually...")

        # Fallback: Create template manually
        govt_template_wb = Workbook()

        # Remove default sheet
        if 'Sheet' in govt_template_wb.sheetnames:
            govt_template_wb.remove(govt_template_wb['Sheet'])

        # Create required sheets
        ws_deductor = govt_template_wb.create_sheet("DEDUCTOR DETAILS")
        ws_challan = govt_template_wb.create_sheet("CHALLAN DETAILS")
        ws_deductee = govt_template_wb.create_sheet("DEDUCTEE BREAK-UP")

        # Fill with our function that now uses Cell 5's builder
        filled_template_wb = fill_government_template(govt_template_wb, updated_masters_data, challan_data_list)

    # Step 6: Save government template
    output_filename = get_output_filename_from_masters(updated_masters_data)
    govt_template_path = os.path.join(output_folder, output_filename)

    filled_template_wb.save(govt_template_path)
    print(f"✅ Generated government template: {output_filename}")

    # Step 7: Generate reconciliation report
    print("\n📊 STEP 6: Generating Reconciliation Report...")
    print("-" * 50)

    report_path = None
    if validation_results:
        try:
            report = ReconciliationReport(
                validation_results=validation_results,
                masters_data=updated_masters_data,
                challan_data=challan_data_list
            )
            report_filename = f"Reconciliation_{output_filename}"
            report_path = report.generate_report(os.path.join(output_folder, report_filename))
            print(f"✅ Generated reconciliation report: {report_filename}")
        except Exception as e:
            print(f"⚠️ ReconciliationReport not available: {str(e)}")
            print("   Skipping report generation...")

    # Step 8: Final validation (existing function)
    print("\n🔍 STEP 7: Final Validation...")
    print("-" * 50)
    final_validation = validate_tds_totals(updated_masters_data, challan_data_list)
    print("✅ Final Validation: " + ("PASSED" if final_validation else "FAILED"))

    # Return all outputs
    print("\n" + "="*60)
    print("✅ ENHANCED TDS PROCESSING COMPLETE!")
    print("="*60)

    results = {
        'govt_template_path': govt_template_path,
        'reconciliation_report_path': report_path,
        'updated_masters_path': updated_masters_data['file_path'],
        'validation_results': validation_results,
        'final_validation': final_validation,
        'challans_processed': len(challan_data_list),
        'output_filename': output_filename
    }

    print("\n📁 Generated Files:")
    print(f"   1. Government Template: {output_filename}")
    if report_path:
        print(f"   2. Reconciliation Report: {os.path.basename(report_path)}")
    print(f"   3. Updated Masters: {os.path.basename(updated_masters_data['file_path'])}")

    return results

# Backwards compatibility wrapper
def process_tds_returns(pdf_folder_path, masters_file_path, template_file_path=None):
    """
    Wrapper for backwards compatibility - ignores template_file_path
    Uses process_tds_returns_enhanced internally
    """
    if template_file_path:
        print("⚠️ Note: Template file no longer needed - generating internally")

    return process_tds_returns_enhanced(pdf_folder_path, masters_file_path)

print("✅ Enhanced Main Processing Pipeline loaded!")
print("\n📌 Key Features:")
print("   ✅ NO TEMPLATE UPLOAD NEEDED - Generated internally")
print("   ✅ Uses GovernmentTemplateBuilder from Cell 5")
print("   ✅ 20 mandatory fields validation (warn-only)")
print("   ✅ Reconciliation report with validation details")
print("   ✅ All critical business logic preserved")
print("   ✅ Backwards compatible with old function signature")
print("   ✅ FIXED: No forward dependencies - properly imports from Cell 5")
print("\n💡 Usage:")
print("   results = process_tds_returns_enhanced(pdf_folder, masters_file)")
print("   # or for backwards compatibility:")
print("   results = process_tds_returns(pdf_folder, masters_file)")
print("\n📁 Outputs:")
print("   - TDS_Month_Year.xlsx (Government format)")
print("   - Reconciliation_TDS_Month_Year.xlsx (Validation report)")
print("   - TDS_Masters_UPDATED.xlsx (Original with updates)")
print("\n⚠️ Prerequisites:")
print("   - Cell 5 must be run first (contains GovernmentTemplateBuilder)")
print("   - Cell 1-4 for basic functions (extract_all_challans, read_tds_masters, etc.)")

✅ Enhanced Main Processing Pipeline loaded!

📌 Key Features:
   ✅ NO TEMPLATE UPLOAD NEEDED - Generated internally
   ✅ Uses GovernmentTemplateBuilder from Cell 5
   ✅ 20 mandatory fields validation (warn-only)
   ✅ Reconciliation report with validation details
   ✅ All critical business logic preserved
   ✅ Backwards compatible with old function signature
   ✅ FIXED: No forward dependencies - properly imports from Cell 5

💡 Usage:
   results = process_tds_returns_enhanced(pdf_folder, masters_file)
   # or for backwards compatibility:
   results = process_tds_returns(pdf_folder, masters_file)

📁 Outputs:
   - TDS_Month_Year.xlsx (Government format)
   - Reconciliation_TDS_Month_Year.xlsx (Validation report)
   - TDS_Masters_UPDATED.xlsx (Original with updates)

⚠️ Prerequisites:
   - Cell 5 must be run first (contains GovernmentTemplateBuilder)
   - Cell 1-4 for basic functions (extract_all_challans, read_tds_masters, etc.)


In [7]:
# Cell 9: Validation Engine with Reconciliation Report Support
# Validates all 20 mandatory fields and prepares data for reconciliation report
# Warn-only behavior - flags issues but doesn't block processing

import re
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from datetime import datetime
from decimal import Decimal, ROUND_HALF_UP
import numpy as np

class ValidationEngine:
    """
    Comprehensive validation engine for TDS Returns
    Validates 20 mandatory fields and performs data reconciliation
    Results are structured for Reconciliation Report generation
    """

    def __init__(self, masters_file_path=None, masters_data=None, challan_data_list=None):
        """
        Initialize validation engine

        Args:
            masters_file_path: Path to TDS_Masters.xlsx file
            masters_data: Already loaded masters data dictionary
            challan_data_list: List of extracted challan data
        """
        self.masters_file_path = masters_file_path
        self.masters_data = masters_data
        self.challan_data_list = challan_data_list or []

        # Define all 20 mandatory fields with their locations
        self.mandatory_fields = {
            # Deductor Information (12 fields)
            'TAN': {'cell': 'B4', 'section': 'deductor', 'format': 'TAN', 'required': True},
            'PAN_DEDUCTOR': {'cell': 'B5', 'section': 'deductor', 'format': 'PAN', 'required': True},
            'FINANCIAL_YEAR': {'cell': 'B6', 'section': 'deductor', 'format': None, 'required': True},
            'ASSESSMENT_YEAR': {'cell': 'B7', 'section': 'deductor', 'format': None, 'required': True},
            'QUARTER': {'cell': 'B8', 'section': 'deductor', 'format': 'QUARTER', 'required': True},
            'STATEMENT_FILED': {'cell': 'B9', 'section': 'deductor', 'format': 'YES_NO', 'required': True},
            'NAME_DEDUCTOR': {'cell': 'B10', 'section': 'deductor', 'format': None, 'required': True},
            'TYPE_DEDUCTOR': {'cell': 'B11', 'section': 'deductor', 'format': 'DEDUCTOR_TYPE', 'required': True},
            'ADDRESS1_DEDUCTOR': {'cell': 'B12', 'section': 'deductor', 'format': None, 'required': True},
            'STATE_DEDUCTOR': {'cell': 'B17', 'section': 'deductor', 'format': None, 'required': True},
            'PINCODE_DEDUCTOR': {'cell': 'B18', 'section': 'deductor', 'format': 'PINCODE', 'required': True},
            'TELEPHONE_DEDUCTOR': {'cell': 'B19', 'section': 'deductor', 'format': None, 'required': True},
            'EMAIL_DEDUCTOR': {'cell': 'B22', 'section': 'deductor', 'format': 'EMAIL', 'required': True},

            # Responsible Person Information (8 fields)
            'NAME_RESPONSIBLE': {'cell': 'B26', 'section': 'responsible', 'format': None, 'required': True},
            'ADDRESS1_RESPONSIBLE': {'cell': 'B28', 'section': 'responsible', 'format': None, 'required': True},
            'STATE_RESPONSIBLE': {'cell': 'B33', 'section': 'responsible', 'format': None, 'required': True},
            'PINCODE_RESPONSIBLE': {'cell': 'B34', 'section': 'responsible', 'format': 'PINCODE', 'required': True},
            'TELEPHONE_RESPONSIBLE': {'cell': 'B35', 'section': 'responsible', 'format': None, 'required': True},
            'EMAIL_RESPONSIBLE': {'cell': 'B38', 'section': 'responsible', 'format': 'EMAIL', 'required': True},
            'TOKEN_NO': {'cell': 'B41', 'section': 'responsible', 'format': None, 'required': True},
            'PAN_RESPONSIBLE': {'cell': 'B42', 'section': 'responsible', 'format': 'PAN', 'required': True},
        }

        # Initialize validation results structure
        self.validation_results = {
            'summary': {
                'total_mandatory_fields': 20,
                'fields_present': 0,
                'fields_missing': 0,
                'fields_valid': 0,
                'fields_invalid': 0,
                'validation_status': 'NOT_RUN',
                'can_proceed': True,  # Warn-only behavior
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            },
            'mandatory_fields': {},
            'format_validations': {},
            'amount_reconciliation': {},
            'errors': [],
            'warnings': [],
            'suggestions': []
        }

    def validate_all(self):
        """
        Run all validations
        Returns: Dictionary with complete validation results
        """
        print("\n🔍 Starting Comprehensive Validation...")
        print("=" * 60)

        # Step 1: Load and validate INPUT-DEDUCTOR DETAILS
        self._load_deductor_details()

        # Step 2: Validate mandatory fields
        self._validate_mandatory_fields()

        # Step 3: Validate formats
        self._validate_formats()

        # Step 4: Reconcile amounts if data available
        if self.masters_data and self.challan_data_list:
            self._reconcile_amounts()

        # Step 5: Generate summary
        self._generate_summary()

        # Step 6: Generate suggestions
        self._generate_suggestions()

        print("\n✅ Validation Complete!")
        print(f"   Status: {self.validation_results['summary']['validation_status']}")
        print(f"   Can Proceed: {'Yes' if self.validation_results['summary']['can_proceed'] else 'No'}")

        return self.validation_results

    def _load_deductor_details(self):
        """Load INPUT-DEDUCTOR DETAILS sheet"""
        try:
            if self.masters_file_path:
                wb = load_workbook(self.masters_file_path, data_only=True)
                if 'INPUT-DEDUCTOR DETAILS' in wb.sheetnames:
                    self.deductor_sheet = wb['INPUT-DEDUCTOR DETAILS']
                    print("✅ Loaded INPUT-DEDUCTOR DETAILS sheet")
                else:
                    self.validation_results['errors'].append("INPUT-DEDUCTOR DETAILS sheet not found")
                    print("❌ INPUT-DEDUCTOR DETAILS sheet not found")
                    self.deductor_sheet = None
            else:
                self.deductor_sheet = None
                self.validation_results['warnings'].append("Masters file path not provided")
        except Exception as e:
            self.validation_results['errors'].append(f"Error loading masters file: {str(e)}")
            self.deductor_sheet = None

    def _validate_mandatory_fields(self):
        """Check presence of all 20 mandatory fields"""
        print("\n📋 Validating Mandatory Fields...")

        if not self.deductor_sheet:
            self.validation_results['warnings'].append("Cannot validate mandatory fields - sheet not loaded")
            return

        fields_present = 0
        fields_missing = 0

        for field_name, field_info in self.mandatory_fields.items():
            cell_value = self.deductor_sheet[field_info['cell']].value

            # Check if field is present
            is_present = cell_value is not None and str(cell_value).strip() != ''

            self.validation_results['mandatory_fields'][field_name] = {
                'cell': field_info['cell'],
                'section': field_info['section'],
                'value': str(cell_value) if cell_value else '',
                'is_present': is_present,
                'is_valid': None,  # Will be set by format validation
                'format_type': field_info['format'],
                'error': None
            }

            if is_present:
                fields_present += 1
            else:
                fields_missing += 1
                if field_info['required']:
                    self.validation_results['warnings'].append(f"Mandatory field missing: {field_name}")

        self.validation_results['summary']['fields_present'] = fields_present
        self.validation_results['summary']['fields_missing'] = fields_missing

        print(f"   ✅ Fields Present: {fields_present}/20")
        print(f"   ⚠️  Fields Missing: {fields_missing}/20")

    def _validate_formats(self):
        """Validate format of fields that have specific format requirements"""
        print("\n🔤 Validating Field Formats...")

        format_validators = {
            'TAN': self._validate_tan,
            'PAN': self._validate_pan,
            'EMAIL': self._validate_email,
            'PINCODE': self._validate_pincode,
            'QUARTER': self._validate_quarter,
            'YES_NO': self._validate_yes_no,
            'DEDUCTOR_TYPE': self._validate_deductor_type
        }

        fields_valid = 0
        fields_invalid = 0

        for field_name, field_data in self.validation_results['mandatory_fields'].items():
            if field_data['is_present'] and field_data['format_type']:
                validator = format_validators.get(field_data['format_type'])
                if validator:
                    is_valid, error_msg = validator(field_data['value'])
                    field_data['is_valid'] = is_valid
                    field_data['error'] = error_msg

                    if is_valid:
                        fields_valid += 1
                    else:
                        fields_invalid += 1
                        self.validation_results['warnings'].append(
                            f"Format error in {field_name}: {error_msg}"
                        )
                else:
                    field_data['is_valid'] = True
                    fields_valid += 1
            elif field_data['is_present']:
                field_data['is_valid'] = True
                fields_valid += 1

        self.validation_results['summary']['fields_valid'] = fields_valid
        self.validation_results['summary']['fields_invalid'] = fields_invalid

        print(f"   ✅ Valid Formats: {fields_valid}")
        print(f"   ❌ Invalid Formats: {fields_invalid}")

    def _validate_tan(self, value):
        """Validate TAN format: AAAA99999A"""
        pattern = r'^[A-Z]{4}\d{5}[A-Z]$'
        if re.match(pattern, str(value).upper()):
            return True, None
        return False, "TAN should be in format AAAA99999A (4 letters, 5 digits, 1 letter)"

    def _validate_pan(self, value):
        """Validate PAN format: AAAAA9999A"""
        pattern = r'^[A-Z]{5}\d{4}[A-Z]$'
        if re.match(pattern, str(value).upper()):
            return True, None
        return False, "PAN should be in format AAAAA9999A (5 letters, 4 digits, 1 letter)"

    def _validate_email(self, value):
        """Validate email format"""
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if re.match(pattern, str(value)):
            return True, None
        return False, "Invalid email format"

    def _validate_pincode(self, value):
        """Validate pincode: 6 digits"""
        pattern = r'^\d{6}$'
        if re.match(pattern, str(value)):
            return True, None
        return False, "Pincode should be 6 digits"

    def _validate_quarter(self, value):
        """Validate quarter: Q1, Q2, Q3, Q4"""
        valid_quarters = ['Q1', 'Q2', 'Q3', 'Q4']
        if str(value).upper() in valid_quarters:
            return True, None
        return False, "Quarter should be Q1, Q2, Q3, or Q4"

    def _validate_yes_no(self, value):
        """Validate YES/NO field"""
        valid_values = ['YES', 'NO']
        if str(value).upper() in valid_values:
            return True, None
        return False, "Value should be YES or NO"

    def _validate_deductor_type(self, value):
        """Validate deductor type: 01-Company or 02-Others"""
        valid_types = ['01-Company', '02-Others', '01', '02']
        if str(value) in valid_types:
            return True, None
        return False, "Deductor type should be 01-Company or 02-Others"

    def _reconcile_amounts(self):
        """Reconcile party-wise TDS totals with challan amounts"""
        print("\n💰 Reconciling Amounts...")

        try:
            if not self.masters_data or not self.challan_data_list:
                self.validation_results['warnings'].append("Cannot reconcile - data not available")
                return

            # Get TDS parties data
            tds_parties = self.masters_data.get('tds_parties')
            code_to_column = self.masters_data.get('code_to_column_name', {})

            if tds_parties is None:
                self.validation_results['warnings'].append("TDS parties data not found")
                return

            # Get column names
            payment_col = code_to_column.get('(415A)')  # Nature of payment
            tds_col = code_to_column.get('(421)')  # TDS amount

            if not payment_col or not tds_col:
                self.validation_results['warnings'].append("Required columns not found for reconciliation")
                return

            # Calculate party totals by nature of payment
            party_totals = {}
            for _, row in tds_parties.iterrows():
                payment_type = str(row.get(payment_col, '')).strip().replace(' ', '')
                if payment_type and payment_type not in ['nan', 'NaT', '']:
                    tds_amount = 0
                    val = row.get(tds_col)
                    if pd.notna(val):
                        tds_amount = Decimal(str(val)).quantize(Decimal('1.'), rounding=ROUND_HALF_UP)

                    if payment_type not in party_totals:
                        party_totals[payment_type] = Decimal('0')
                    party_totals[payment_type] += tds_amount

            # Calculate challan totals
            challan_totals = {}
            for challan in self.challan_data_list:
                nop = challan.get('nature_of_payment', '').replace(' ', '')
                tax_amount = Decimal(str(challan.get('tax_amount', 0))).quantize(
                    Decimal('1.'), rounding=ROUND_HALF_UP
                )
                if nop:
                    if nop not in challan_totals:
                        challan_totals[nop] = Decimal('0')
                    challan_totals[nop] += tax_amount

            # Compare and create reconciliation data
            all_nops = sorted(set(list(party_totals.keys()) + list(challan_totals.keys())))
            reconciliation_data = []
            total_matched = 0
            total_mismatched = 0

            for nop in all_nops:
                party_total = party_totals.get(nop, Decimal('0'))
                challan_total = challan_totals.get(nop, Decimal('0'))
                difference = abs(party_total - challan_total)

                status = 'MATCHED' if difference <= 1 else 'MISMATCHED'
                if status == 'MATCHED':
                    total_matched += 1
                else:
                    total_mismatched += 1
                    self.validation_results['warnings'].append(
                        f"Amount mismatch for {nop}: Party Total ₹{party_total:,.0f} vs Challan ₹{challan_total:,.0f}"
                    )

                reconciliation_data.append({
                    'nature_of_payment': nop,
                    'party_total': float(party_total),
                    'challan_total': float(challan_total),
                    'difference': float(difference),
                    'status': status
                })

            self.validation_results['amount_reconciliation'] = {
                'details': reconciliation_data,
                'summary': {
                    'total_sections': len(all_nops),
                    'matched': total_matched,
                    'mismatched': total_mismatched,
                    'party_grand_total': float(sum(party_totals.values())),
                    'challan_grand_total': float(sum(challan_totals.values()))
                }
            }

            print(f"   ✅ Matched Sections: {total_matched}")
            print(f"   ❌ Mismatched Sections: {total_mismatched}")
            print(f"   Party Total: ₹{sum(party_totals.values()):,.0f}")
            print(f"   Challan Total: ₹{sum(challan_totals.values()):,.0f}")

        except Exception as e:
            self.validation_results['errors'].append(f"Error during reconciliation: {str(e)}")
            print(f"   ❌ Reconciliation error: {str(e)}")

    def _generate_summary(self):
        """Generate validation summary"""
        summary = self.validation_results['summary']

        # Determine overall status
        if summary['fields_missing'] == 0 and summary['fields_invalid'] == 0:
            summary['validation_status'] = 'PASS'
        elif summary['fields_missing'] > 10 or len(self.validation_results['errors']) > 0:
            summary['validation_status'] = 'CRITICAL'
        else:
            summary['validation_status'] = 'WARNING'

        # Can always proceed (warn-only behavior)
        summary['can_proceed'] = True

        print("\n📊 Validation Summary:")
        print(f"   Overall Status: {summary['validation_status']}")
        print(f"   Mandatory Fields: {summary['fields_present']}/{summary['total_mandatory_fields']}")
        print(f"   Format Validations: {summary['fields_valid']} valid, {summary['fields_invalid']} invalid")
        print(f"   Errors: {len(self.validation_results['errors'])}")
        print(f"   Warnings: {len(self.validation_results['warnings'])}")

    def _generate_suggestions(self):
        """Generate suggestions for fixing validation issues"""
        suggestions = []

        # Check missing mandatory fields
        missing_deductor = []
        missing_responsible = []

        for field_name, field_data in self.validation_results['mandatory_fields'].items():
            if not field_data['is_present']:
                if field_data['section'] == 'deductor':
                    missing_deductor.append(field_name)
                else:
                    missing_responsible.append(field_name)

        if missing_deductor:
            suggestions.append(f"Complete {len(missing_deductor)} missing deductor fields")
        if missing_responsible:
            suggestions.append(f"Complete {len(missing_responsible)} missing responsible person fields")

        # Check format errors
        format_errors = [
            field for field, data in self.validation_results['mandatory_fields'].items()
            if data['is_present'] and data['is_valid'] == False
        ]

        if format_errors:
            suggestions.append(f"Fix format errors in {len(format_errors)} fields")

        # Check amount mismatches
        if self.validation_results.get('amount_reconciliation'):
            mismatched = self.validation_results['amount_reconciliation']['summary'].get('mismatched', 0)
            if mismatched > 0:
                suggestions.append(f"Review {mismatched} payment sections with amount mismatches")

        self.validation_results['suggestions'] = suggestions

        if suggestions:
            print("\n💡 Suggestions:")
            for i, suggestion in enumerate(suggestions, 1):
                print(f"   {i}. {suggestion}")

    def get_validation_dataframe(self):
        """
        Convert validation results to DataFrame for easy viewing
        Returns: DataFrame with validation results
        """
        if not self.validation_results['mandatory_fields']:
            return pd.DataFrame()

        data = []
        for field_name, field_info in self.validation_results['mandatory_fields'].items():
            data.append({
                'Field Name': field_name,
                'Section': field_info['section'].title(),
                'Value': field_info['value'][:30] if field_info['value'] else 'MISSING',
                'Present': '✅' if field_info['is_present'] else '❌',
                'Valid': '✅' if field_info['is_valid'] else '❌' if field_info['is_valid'] == False else '-',
                'Error': field_info['error'] if field_info['error'] else ''
            })

        return pd.DataFrame(data)

    def export_validation_report(self, output_path='Validation_Report.xlsx'):
        """
        Export validation results to Excel file
        This will be used by ReconciliationReport class
        """
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            # Sheet 1: Summary
            summary_df = pd.DataFrame([self.validation_results['summary']])
            summary_df.to_excel(writer, sheet_name='Summary', index=False)

            # Sheet 2: Mandatory Fields
            fields_df = self.get_validation_dataframe()
            fields_df.to_excel(writer, sheet_name='Mandatory_Fields', index=False)

            # Sheet 3: Amount Reconciliation
            if self.validation_results.get('amount_reconciliation'):
                recon_df = pd.DataFrame(self.validation_results['amount_reconciliation']['details'])
                recon_df.to_excel(writer, sheet_name='Amount_Reconciliation', index=False)

            # Sheet 4: Errors and Warnings
            issues_df = pd.DataFrame({
                'Type': ['Error'] * len(self.validation_results['errors']) +
                        ['Warning'] * len(self.validation_results['warnings']),
                'Message': self.validation_results['errors'] + self.validation_results['warnings']
            })
            if not issues_df.empty:
                issues_df.to_excel(writer, sheet_name='Issues', index=False)

        print(f"\n📊 Validation report exported to: {output_path}")
        return output_path


# Test function for validation
def test_validation(masters_file_path, challan_data_list=None):
    """
    Test the validation engine

    Args:
        masters_file_path: Path to TDS_Masters.xlsx
        challan_data_list: Optional list of challan data
    """
    print("🧪 Testing Validation Engine...")

    # Initialize validator
    validator = ValidationEngine(
        masters_file_path=masters_file_path,
        challan_data_list=challan_data_list
    )

    # Run validation
    results = validator.validate_all()

    # Display results as DataFrame
    print("\n📋 Mandatory Fields Status:")
    df = validator.get_validation_dataframe()
    print(df.to_string())

    # Export report
    report_path = validator.export_validation_report('TDS_Validation_Report.xlsx')

    return results, report_path


print("✅ Cell 9: Validation Engine loaded successfully!")
print("\n📋 Features:")
print("   - Validates all 20 mandatory fields")
print("   - Checks TAN, PAN, Email, Pincode formats")
print("   - Reconciles party vs challan amounts")
print("   - Warn-only behavior (doesn't block processing)")
print("   - Generates detailed validation report")
print("\n💡 Usage:")
print("   validator = ValidationEngine(masters_file_path='TDS_Masters.xlsx')")
print("   results = validator.validate_all()")
print("\n📊 The validation results are structured for Reconciliation Report generation")

✅ Cell 9: Validation Engine loaded successfully!

📋 Features:
   - Validates all 20 mandatory fields
   - Checks TAN, PAN, Email, Pincode formats
   - Reconciles party vs challan amounts
   - Warn-only behavior (doesn't block processing)
   - Generates detailed validation report

💡 Usage:
   validator = ValidationEngine(masters_file_path='TDS_Masters.xlsx')
   results = validator.validate_all()

📊 The validation results are structured for Reconciliation Report generation


In [8]:
# Cell 10: Reconciliation Report Generator
# Creates comprehensive Excel report with validation results and reconciliation details
# This enhanced report helps users understand and fix validation issues

import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.utils import get_column_letter
from openpyxl.formatting.rule import CellIsRule
from openpyxl.styles.differential import DifferentialStyle
from datetime import datetime
import numpy as np

class ReconciliationReport:
    """
    Generate comprehensive reconciliation report with validation results
    Creates multi-sheet Excel workbook with formatted results
    """

    def __init__(self, validation_results, masters_data=None, challan_data=None):
        """
        Initialize report generator

        Args:
            validation_results: Results from ValidationEngine
            masters_data: Original masters data
            challan_data: Challan extraction results
        """
        self.validation_results = validation_results
        self.masters_data = masters_data
        self.challan_data = challan_data
        self.wb = None

        # Define color schemes
        self.colors = {
            'header': 'CCE5FF',  # Light blue
            'pass': 'C6EFCE',    # Light green
            'fail': 'FFC7CE',    # Light red
            'warning': 'FFEB9C', # Light yellow
            'info': 'F2F2F2',    # Light gray
            'critical': 'FF0000', # Red text
            'success': '008000'  # Green text
        }

    def generate_report(self, output_filename='TDS_Reconciliation_Report.xlsx'):
        """
        Generate complete reconciliation report

        Args:
            output_filename: Name of output Excel file

        Returns:
            Path to generated report
        """
        print("\n📊 Generating Reconciliation Report...")
        print("=" * 60)

        # Create new workbook
        self.wb = Workbook()

        # Remove default sheet
        if 'Sheet' in self.wb.sheetnames:
            self.wb.remove(self.wb['Sheet'])

        # Create all report sheets
        self._create_summary_sheet()
        self._create_mandatory_fields_sheet()
        self._create_challan_matching_sheet()
        self._create_validation_errors_sheet()
        self._create_data_corrections_sheet()

        # Save the report
        self.wb.save(output_filename)
        print(f"\n✅ Report generated: {output_filename}")

        # Print summary
        self._print_report_summary()

        return output_filename

    def _create_summary_sheet(self):
        """Create SUMMARY dashboard sheet"""
        ws = self.wb.create_sheet("SUMMARY")

        # Title
        ws['A1'] = "TDS RECONCILIATION REPORT - SUMMARY"
        ws['A1'].font = Font(bold=True, size=16, color="0000FF")
        ws.merge_cells('A1:F1')
        ws['A1'].alignment = Alignment(horizontal='center')

        # Report metadata
        ws['A3'] = "Report Generated:"
        ws['B3'] = datetime.now().strftime('%d-%m-%Y %H:%M:%S')
        ws['A4'] = "Validation Status:"
        status = self.validation_results['summary'].get('validation_status', 'UNKNOWN')
        ws['B4'] = status

        # Apply color based on status
        if status == 'PASS':
            ws['B4'].fill = PatternFill(start_color=self.colors['pass'], fill_type='solid')
        elif status == 'WARNING':
            ws['B4'].fill = PatternFill(start_color=self.colors['warning'], fill_type='solid')
        else:
            ws['B4'].fill = PatternFill(start_color=self.colors['fail'], fill_type='solid')

        # Mandatory Fields Summary
        ws['A6'] = "MANDATORY FIELDS SUMMARY"
        ws['A6'].font = Font(bold=True, size=12, underline='single')
        ws.merge_cells('A6:D6')

        summary_data = [
            ("Total Mandatory Fields", self.validation_results['summary']['total_mandatory_fields']),
            ("Fields Present", self.validation_results['summary']['fields_present']),
            ("Fields Missing", self.validation_results['summary']['fields_missing']),
            ("Fields with Valid Format", self.validation_results['summary']['fields_valid']),
            ("Fields with Invalid Format", self.validation_results['summary']['fields_invalid'])
        ]

        row = 7
        for label, value in summary_data:
            ws[f'A{row}'] = label
            ws[f'B{row}'] = value

            # Add visual indicator
            if 'Missing' in label and value > 0:
                ws[f'B{row}'].fill = PatternFill(start_color=self.colors['fail'], fill_type='solid')
            elif 'Invalid' in label and value > 0:
                ws[f'B{row}'].fill = PatternFill(start_color=self.colors['warning'], fill_type='solid')
            elif 'Valid' in label or 'Present' in label:
                ws[f'B{row}'].fill = PatternFill(start_color=self.colors['pass'], fill_type='solid')

            row += 1

        # Amount Reconciliation Summary
        ws['A13'] = "AMOUNT RECONCILIATION SUMMARY"
        ws['A13'].font = Font(bold=True, size=12, underline='single')
        ws.merge_cells('A13:D13')

        if self.validation_results.get('amount_reconciliation'):
            recon_summary = self.validation_results['amount_reconciliation']['summary']

            recon_data = [
                ("Total Payment Sections", recon_summary.get('total_sections', 0)),
                ("Matched Sections", recon_summary.get('matched', 0)),
                ("Mismatched Sections", recon_summary.get('mismatched', 0)),
                ("Party Grand Total (₹)", f"{recon_summary.get('party_grand_total', 0):,.2f}"),
                ("Challan Grand Total (₹)", f"{recon_summary.get('challan_grand_total', 0):,.2f}")
            ]

            row = 14
            for label, value in recon_data:
                ws[f'A{row}'] = label
                ws[f'B{row}'] = value

                if 'Mismatched' in label and value > 0:
                    ws[f'B{row}'].fill = PatternFill(start_color=self.colors['fail'], fill_type='solid')
                elif 'Matched' in label:
                    ws[f'B{row}'].fill = PatternFill(start_color=self.colors['pass'], fill_type='solid')

                row += 1
        else:
            ws['A14'] = "No amount reconciliation data available"

        # Issues Summary
        ws['A20'] = "ISSUES SUMMARY"
        ws['A20'].font = Font(bold=True, size=12, underline='single')
        ws.merge_cells('A20:D20')

        ws['A21'] = "Total Errors:"
        ws['B21'] = len(self.validation_results.get('errors', []))
        ws['A22'] = "Total Warnings:"
        ws['B22'] = len(self.validation_results.get('warnings', []))

        if ws['B21'].value > 0:
            ws['B21'].fill = PatternFill(start_color=self.colors['fail'], fill_type='solid')
        if ws['B22'].value > 0:
            ws['B22'].fill = PatternFill(start_color=self.colors['warning'], fill_type='solid')

        # Processing Recommendation
        ws['A25'] = "PROCESSING RECOMMENDATION"
        ws['A25'].font = Font(bold=True, size=12, underline='single')
        ws.merge_cells('A25:D25')

        can_proceed = self.validation_results['summary'].get('can_proceed', False)
        ws['A26'] = "Can Proceed with Processing:"
        ws['B26'] = "YES" if can_proceed else "NO"
        ws['B26'].font = Font(bold=True, color=self.colors['success'] if can_proceed else self.colors['critical'])

        # Add suggestions if any
        suggestions = self.validation_results.get('suggestions', [])
        if suggestions:
            ws['A28'] = "RECOMMENDATIONS:"
            ws['A28'].font = Font(bold=True, size=12)
            row = 29
            for i, suggestion in enumerate(suggestions, 1):
                ws[f'A{row}'] = f"{i}. {suggestion}"
                row += 1

        # Format column widths
        ws.column_dimensions['A'].width = 30
        ws.column_dimensions['B'].width = 20
        ws.column_dimensions['C'].width = 20
        ws.column_dimensions['D'].width = 20

        print("   ✅ Summary sheet created")

    def _create_mandatory_fields_sheet(self):
        """Create MANDATORY_FIELDS detailed sheet"""
        ws = self.wb.create_sheet("MANDATORY_FIELDS")

        # Title
        ws['A1'] = "MANDATORY FIELDS VALIDATION DETAILS"
        ws['A1'].font = Font(bold=True, size=14)
        ws.merge_cells('A1:G1')
        ws['A1'].alignment = Alignment(horizontal='center')

        # Create headers
        headers = [
            "S.No", "Field Name", "Section", "Required",
            "Present", "Format Valid", "Current Value", "Error/Issue"
        ]

        for col, header in enumerate(headers, 1):
            cell = ws.cell(row=3, column=col)
            cell.value = header
            cell.font = Font(bold=True, color="FFFFFF")
            cell.fill = PatternFill(start_color="366092", fill_type='solid')
            cell.alignment = Alignment(horizontal='center')

        # Add data
        row = 4
        for idx, (field_name, field_data) in enumerate(
            self.validation_results.get('mandatory_fields', {}).items(), 1
        ):
            ws.cell(row=row, column=1).value = idx
            ws.cell(row=row, column=2).value = field_name
            ws.cell(row=row, column=3).value = field_data['section'].title()
            ws.cell(row=row, column=4).value = "YES"  # All 20 are required

            # Present status
            is_present = field_data['is_present']
            ws.cell(row=row, column=5).value = "YES" if is_present else "NO"
            if is_present:
                ws.cell(row=row, column=5).fill = PatternFill(start_color=self.colors['pass'], fill_type='solid')
            else:
                ws.cell(row=row, column=5).fill = PatternFill(start_color=self.colors['fail'], fill_type='solid')

            # Format valid status
            is_valid = field_data.get('is_valid')
            if is_valid is not None:
                ws.cell(row=row, column=6).value = "YES" if is_valid else "NO"
                if is_valid:
                    ws.cell(row=row, column=6).fill = PatternFill(start_color=self.colors['pass'], fill_type='solid')
                else:
                    ws.cell(row=row, column=6).fill = PatternFill(start_color=self.colors['fail'], fill_type='solid')
            else:
                ws.cell(row=row, column=6).value = "N/A"

            # Current value
            value = field_data.get('value', '')
            ws.cell(row=row, column=7).value = value[:50] if value else "MISSING"

            # Error message
            error = field_data.get('error', '')
            ws.cell(row=row, column=8).value = error if error else ("Field is missing" if not is_present else "")
            if error or not is_present:
                ws.cell(row=row, column=8).font = Font(color=self.colors['critical'])

            row += 1

        # Add section summaries
        row += 2
        ws.cell(row=row, column=1).value = "SECTION SUMMARY"
        ws.cell(row=row, column=1).font = Font(bold=True, size=12)
        row += 1

        # Count by section
        deductor_present = sum(
            1 for f in self.validation_results.get('mandatory_fields', {}).values()
            if f['section'] == 'deductor' and f['is_present']
        )
        responsible_present = sum(
            1 for f in self.validation_results.get('mandatory_fields', {}).values()
            if f['section'] == 'responsible' and f['is_present']
        )

        ws.cell(row=row, column=1).value = "Deductor Fields:"
        ws.cell(row=row, column=2).value = f"{deductor_present}/12"
        row += 1
        ws.cell(row=row, column=1).value = "Responsible Person Fields:"
        ws.cell(row=row, column=2).value = f"{responsible_present}/8"

        # Format column widths
        ws.column_dimensions['A'].width = 8
        ws.column_dimensions['B'].width = 25
        ws.column_dimensions['C'].width = 15
        ws.column_dimensions['D'].width = 10
        ws.column_dimensions['E'].width = 10
        ws.column_dimensions['F'].width = 15
        ws.column_dimensions['G'].width = 40
        ws.column_dimensions['H'].width = 40

        print("   ✅ Mandatory fields sheet created")

    def _create_challan_matching_sheet(self):
        """Create CHALLAN_MATCHING sheet with amount reconciliation"""
        ws = self.wb.create_sheet("CHALLAN_MATCHING")

        # Title
        ws['A1'] = "CHALLAN VS PARTY AMOUNT RECONCILIATION"
        ws['A1'].font = Font(bold=True, size=14)
        ws.merge_cells('A1:F1')
        ws['A1'].alignment = Alignment(horizontal='center')

        if self.validation_results.get('amount_reconciliation'):
            # Headers
            headers = [
                "S.No", "Nature of Payment", "Party Total (₹)",
                "Challan Total (₹)", "Difference (₹)", "Status"
            ]

            for col, header in enumerate(headers, 1):
                cell = ws.cell(row=3, column=col)
                cell.value = header
                cell.font = Font(bold=True, color="FFFFFF")
                cell.fill = PatternFill(start_color="366092", fill_type='solid')
                cell.alignment = Alignment(horizontal='center')

            # Add reconciliation data
            row = 4
            recon_details = self.validation_results['amount_reconciliation'].get('details', [])

            for idx, item in enumerate(recon_details, 1):
                ws.cell(row=row, column=1).value = idx
                ws.cell(row=row, column=2).value = item['nature_of_payment']
                ws.cell(row=row, column=3).value = f"{item['party_total']:,.2f}"
                ws.cell(row=row, column=4).value = f"{item['challan_total']:,.2f}"
                ws.cell(row=row, column=5).value = f"{item['difference']:,.2f}"
                ws.cell(row=row, column=6).value = item['status']

                # Color code status
                if item['status'] == 'MATCHED':
                    ws.cell(row=row, column=6).fill = PatternFill(start_color=self.colors['pass'], fill_type='solid')
                else:
                    ws.cell(row=row, column=6).fill = PatternFill(start_color=self.colors['fail'], fill_type='solid')
                    # Highlight the entire row for mismatches
                    for col in range(1, 7):
                        ws.cell(row=row, column=col).fill = PatternFill(start_color=self.colors['warning'], fill_type='solid')

                row += 1

            # Add totals row
            row += 1
            summary = self.validation_results['amount_reconciliation']['summary']
            ws.cell(row=row, column=2).value = "GRAND TOTAL"
            ws.cell(row=row, column=2).font = Font(bold=True)
            ws.cell(row=row, column=3).value = f"{summary['party_grand_total']:,.2f}"
            ws.cell(row=row, column=3).font = Font(bold=True)
            ws.cell(row=row, column=4).value = f"{summary['challan_grand_total']:,.2f}"
            ws.cell(row=row, column=4).font = Font(bold=True)

            diff = abs(summary['party_grand_total'] - summary['challan_grand_total'])
            ws.cell(row=row, column=5).value = f"{diff:,.2f}"
            ws.cell(row=row, column=5).font = Font(bold=True)

            # Add border to totals row
            for col in range(2, 6):
                ws.cell(row=row, column=col).border = Border(
                    top=Side(style='double'),
                    bottom=Side(style='double')
                )
        else:
            ws['A3'] = "No amount reconciliation data available"
            ws['A4'] = "Please ensure both TDS parties data and challan data are provided"

        # Format column widths
        ws.column_dimensions['A'].width = 8
        ws.column_dimensions['B'].width = 20
        ws.column_dimensions['C'].width = 20
        ws.column_dimensions['D'].width = 20
        ws.column_dimensions['E'].width = 20
        ws.column_dimensions['F'].width = 15

        print("   ✅ Challan matching sheet created")

    def _create_validation_errors_sheet(self):
        """Create VALIDATION_ERRORS sheet with all issues"""
        ws = self.wb.create_sheet("VALIDATION_ERRORS")

        # Title
        ws['A1'] = "VALIDATION ERRORS AND WARNINGS"
        ws['A1'].font = Font(bold=True, size=14)
        ws.merge_cells('A1:D1')
        ws['A1'].alignment = Alignment(horizontal='center')

        # Combine errors and warnings
        all_issues = []

        for error in self.validation_results.get('errors', []):
            all_issues.append(('ERROR', error, 'Critical'))

        for warning in self.validation_results.get('warnings', []):
            all_issues.append(('WARNING', warning, 'Review Required'))

        if all_issues:
            # Headers
            headers = ["S.No", "Type", "Issue Description", "Action Required"]

            for col, header in enumerate(headers, 1):
                cell = ws.cell(row=3, column=col)
                cell.value = header
                cell.font = Font(bold=True, color="FFFFFF")
                cell.fill = PatternFill(start_color="366092", fill_type='solid')
                cell.alignment = Alignment(horizontal='center')

            # Add issues
            row = 4
            for idx, (issue_type, message, action) in enumerate(all_issues, 1):
                ws.cell(row=row, column=1).value = idx
                ws.cell(row=row, column=2).value = issue_type
                ws.cell(row=row, column=3).value = message
                ws.cell(row=row, column=4).value = action

                # Color code by type
                if issue_type == 'ERROR':
                    ws.cell(row=row, column=2).fill = PatternFill(start_color=self.colors['fail'], fill_type='solid')
                    ws.cell(row=row, column=2).font = Font(bold=True, color=self.colors['critical'])
                else:
                    ws.cell(row=row, column=2).fill = PatternFill(start_color=self.colors['warning'], fill_type='solid')

                row += 1

            # Add summary
            row += 2
            ws.cell(row=row, column=1).value = "SUMMARY"
            ws.cell(row=row, column=1).font = Font(bold=True, size=12)
            row += 1
            ws.cell(row=row, column=1).value = "Total Errors:"
            ws.cell(row=row, column=2).value = len(self.validation_results.get('errors', []))
            row += 1
            ws.cell(row=row, column=1).value = "Total Warnings:"
            ws.cell(row=row, column=2).value = len(self.validation_results.get('warnings', []))
        else:
            ws['A3'] = "✅ No validation errors or warnings found!"
            ws['A3'].font = Font(color=self.colors['success'], bold=True)

        # Format column widths
        ws.column_dimensions['A'].width = 8
        ws.column_dimensions['B'].width = 15
        ws.column_dimensions['C'].width = 60
        ws.column_dimensions['D'].width = 20

        print("   ✅ Validation errors sheet created")

    def _create_data_corrections_sheet(self):
        """Create DATA_CORRECTIONS sheet with suggested fixes"""
        ws = self.wb.create_sheet("DATA_CORRECTIONS")

        # Title
        ws['A1'] = "SUGGESTED DATA CORRECTIONS"
        ws['A1'].font = Font(bold=True, size=14)
        ws.merge_cells('A1:E1')
        ws['A1'].alignment = Alignment(horizontal='center')

        # Headers
        headers = ["S.No", "Field Name", "Current Value", "Issue", "Suggested Correction"]

        for col, header in enumerate(headers, 1):
            cell = ws.cell(row=3, column=col)
            cell.value = header
            cell.font = Font(bold=True, color="FFFFFF")
            cell.fill = PatternFill(start_color="366092", fill_type='solid')
            cell.alignment = Alignment(horizontal='center')

        # Collect corrections needed
        corrections = []

        # Check mandatory fields
        for field_name, field_data in self.validation_results.get('mandatory_fields', {}).items():
            if not field_data['is_present']:
                corrections.append({
                    'field': field_name,
                    'current': 'MISSING',
                    'issue': 'Mandatory field is missing',
                    'suggestion': f'Please provide {field_name.replace("_", " ").title()}'
                })
            elif field_data.get('is_valid') == False:
                corrections.append({
                    'field': field_name,
                    'current': field_data.get('value', ''),
                    'issue': field_data.get('error', 'Invalid format'),
                    'suggestion': self._get_format_suggestion(field_name, field_data.get('format_type'))
                })

        # Add corrections to sheet
        if corrections:
            row = 4
            for idx, correction in enumerate(corrections, 1):
                ws.cell(row=row, column=1).value = idx
                ws.cell(row=row, column=2).value = correction['field']
                ws.cell(row=row, column=3).value = correction['current'][:50] if correction['current'] else 'MISSING'
                ws.cell(row=row, column=4).value = correction['issue']
                ws.cell(row=row, column=5).value = correction['suggestion']

                # Highlight critical corrections
                if 'MISSING' in correction['current']:
                    for col in range(1, 6):
                        ws.cell(row=row, column=col).fill = PatternFill(start_color=self.colors['warning'], fill_type='solid')

                row += 1

            # Add instructions
            row += 2
            ws.cell(row=row, column=1).value = "INSTRUCTIONS:"
            ws.cell(row=row, column=1).font = Font(bold=True, size=12)
            row += 1

            instructions = [
                "1. Review all suggested corrections above",
                "2. Update the TDS_Masters.xlsx file with correct values",
                "3. Ensure all mandatory fields are filled",
                "4. Follow the format requirements for TAN, PAN, Email, and Pincode",
                "5. Re-run the validation after making corrections"
            ]

            for instruction in instructions:
                ws.cell(row=row, column=1).value = instruction
                ws.merge_cells(f'A{row}:E{row}')
                row += 1
        else:
            ws['A3'] = "✅ No corrections needed - all validations passed!"
            ws['A3'].font = Font(color=self.colors['success'], bold=True)

        # Format column widths
        ws.column_dimensions['A'].width = 8
        ws.column_dimensions['B'].width = 25
        ws.column_dimensions['C'].width = 30
        ws.column_dimensions['D'].width = 35
        ws.column_dimensions['E'].width = 40

        print("   ✅ Data corrections sheet created")

    def _get_format_suggestion(self, field_name, format_type):
        """Get specific format suggestion based on field type"""
        suggestions = {
            'TAN': 'Enter in format: AAAA99999A (4 letters, 5 digits, 1 letter)',
            'PAN': 'Enter in format: AAAAA9999A (5 letters, 4 digits, 1 letter)',
            'EMAIL': 'Enter valid email: example@domain.com',
            'PINCODE': 'Enter 6-digit pincode: 123456',
            'QUARTER': 'Enter one of: Q1, Q2, Q3, Q4',
            'YES_NO': 'Enter either YES or NO',
            'DEDUCTOR_TYPE': 'Enter either 01-Company or 02-Others'
        }
        return suggestions.get(format_type, f'Please provide valid {field_name.replace("_", " ").lower()}')

    def _print_report_summary(self):
        """Print summary of generated report"""
        print("\n📊 Report Summary:")
        print("-" * 40)

        summary = self.validation_results.get('summary', {})
        print(f"Overall Status: {summary.get('validation_status', 'UNKNOWN')}")
        print(f"Mandatory Fields: {summary.get('fields_present', 0)}/20 present")
        print(f"Format Validations: {summary.get('fields_valid', 0)} valid, {summary.get('fields_invalid', 0)} invalid")

        if self.validation_results.get('amount_reconciliation'):
            recon = self.validation_results['amount_reconciliation']['summary']
            print(f"Amount Matching: {recon.get('matched', 0)} matched, {recon.get('mismatched', 0)} mismatched")

        print(f"Total Issues: {len(self.validation_results.get('errors', []))} errors, {len(self.validation_results.get('warnings', []))} warnings")
        print("-" * 40)


# Test function
def test_reconciliation_report(validation_results):
    """
    Test the reconciliation report generator

    Args:
        validation_results: Results from ValidationEngine
    """
    print("🧪 Testing Reconciliation Report Generator...")

    # Create report generator
    report = ReconciliationReport(validation_results)

    # Generate report
    report_path = report.generate_report('TDS_Reconciliation_Report.xlsx')

    print(f"\n✅ Report generated successfully: {report_path}")
    return report_path


print("✅ Cell 10: Reconciliation Report Generator loaded successfully!")
print("\n📋 Features:")
print("   - Multi-sheet Excel report with formatting")
print("   - SUMMARY dashboard with overall status")
print("   - MANDATORY_FIELDS detailed validation")
print("   - CHALLAN_MATCHING amount reconciliation")
print("   - VALIDATION_ERRORS comprehensive issue list")
print("   - DATA_CORRECTIONS actionable suggestions")
print("\n💡 Usage:")
print("   report = ReconciliationReport(validation_results)")
print("   report_path = report.generate_report('TDS_Reconciliation_Report.xlsx')")
print("\n📊 This enhanced report helps users understand and fix all validation issues")

✅ Cell 10: Reconciliation Report Generator loaded successfully!

📋 Features:
   - Multi-sheet Excel report with formatting
   - SUMMARY dashboard with overall status
   - MANDATORY_FIELDS detailed validation
   - CHALLAN_MATCHING amount reconciliation
   - VALIDATION_ERRORS comprehensive issue list
   - DATA_CORRECTIONS actionable suggestions

💡 Usage:
   report = ReconciliationReport(validation_results)
   report_path = report.generate_report('TDS_Reconciliation_Report.xlsx')

📊 This enhanced report helps users understand and fix all validation issues


In [10]:
# Cell 9: Enhanced Main Processing Pipeline
# This integrates validation, internal template generation, and reporting
# NO TEMPLATE UPLOAD NEEDED - Government template generated internally
# FIXED: Removed duplicate _fill_ functions, now uses GovernmentTemplateBuilder from Cell 5

import os
import math
import pandas as pd
import numpy as np
from datetime import datetime
from decimal import Decimal, ROUND_HALF_UP
import openpyxl
from openpyxl import load_workbook, Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill

def get_output_filename_from_masters(tds_masters_data):
    """
    Extract month and year from the first payment date in TDS Masters
    to generate output filename as TDS_Month_Year.xlsx
    """
    try:
        code_to_column_name = tds_masters_data.get('code_to_column_name', {})
        date_col = code_to_column_name.get('(418)')
        if date_col and date_col in tds_masters_data['tds_parties'].columns:
            dates = tds_masters_data['tds_parties'][date_col].dropna()
            if not dates.empty:
                first_date = pd.to_datetime(dates.iloc[0])
                month_name = first_date.strftime('%B')
                year = first_date.strftime('%Y')
                return f"TDS_{month_name}_{year}.xlsx"
        current_date = datetime.now()
        return f"TDS_{current_date.strftime('%B')}_{current_date.strftime('%Y')}.xlsx"
    except:
        current_date = datetime.now()
        return f"TDS_{current_date.strftime('%B')}_{current_date.strftime('%Y')}.xlsx"

def fill_government_template(wb, tds_masters_data, challan_data_list):
    """
    Fill the internally generated government template with data
    Now uses GovernmentTemplateBuilder from Cell 5 for the actual filling logic

    Args:
        wb: Workbook object (not used - kept for backward compatibility)
        tds_masters_data: Dictionary containing TDS Masters data
        challan_data_list: List of challan dictionaries from PDF extraction

    Returns:
        Filled workbook object
    """
    print("\n📝 Filling Government Template...")

    # Import the GovernmentTemplateBuilder from Cell 5
    try:
        from GovernmentTemplateBuilder import GovernmentTemplateBuilder
        print("   Using GovernmentTemplateBuilder from Cell 5...")

        # Create builder and fill template
        builder = GovernmentTemplateBuilder()
        filled_wb = builder.fill_template(tds_masters_data, challan_data_list)

        print("✅ Government template filled successfully")
        return filled_wb

    except ImportError:
        print("⚠️ GovernmentTemplateBuilder not found. Please run Cell 5 first.")
        raise ImportError("GovernmentTemplateBuilder class not found. Ensure Cell 5 is executed first.")

def process_tds_returns_enhanced(pdf_folder_path, masters_file_path, output_folder=None):
    """
    Enhanced main processing function - NO TEMPLATE UPLOAD NEEDED
    Integrates validation, internal template generation, and reconciliation report

    Args:
        pdf_folder_path: Path to folder containing PDF challans
        masters_file_path: Path to TDS_Masters.xlsx file
        output_folder: Optional output folder path

    Returns:
        Dictionary with all output paths and validation results
    """
    print("\n" + "="*60)
    print("🚀 STARTING ENHANCED TDS RETURN PROCESSING")
    print("="*60)

    # Set output folder
    if not output_folder:
        output_folder = os.path.dirname(masters_file_path)

    # Step 1: Extract PDFs (existing function)
    print("\n📄 STEP 1: Extracting Challan Data from PDFs...")
    print("-" * 50)
    challan_data_list = extract_all_challans(pdf_folder_path)

    if not challan_data_list:
        print("❌ No valid challans extracted. Aborting.")
        return None

    print(f"✅ Successfully extracted {len(challan_data_list)} unique challans")

    # Step 2: Read Masters (existing function)
    print("\n📊 STEP 2: Reading TDS Masters File...")
    print("-" * 50)
    tds_masters_data = read_tds_masters(masters_file_path)

    if not tds_masters_data:
        print("❌ Failed to read TDS Masters. Aborting.")
        return None

    print("✅ Successfully read TDS Masters file")

    # Step 3: Validate mandatory fields
    print("\n🔍 STEP 3: Validating Mandatory Fields...")
    print("-" * 50)

    try:
        validator = ValidationEngine(
            masters_file_path=masters_file_path,
            masters_data=tds_masters_data,
            challan_data_list=challan_data_list
        )
        validation_results = validator.validate_all()

        # Check validation status but don't block (warn-only)
        validation_status = validation_results['summary']['validation_status']
        if validation_status == 'CRITICAL':
            print("⚠️ Critical validation issues found - proceeding anyway (warn-only mode)")
        elif validation_status == 'WARNING':
            print("⚠️ Minor validation issues found - proceeding")
        else:
            print("✅ All validations passed!")
    except Exception as e:
        print(f"⚠️ Validation engine not available: {str(e)}")
        print("   Proceeding without validation...")
        validation_results = None

    # Step 4: Update Masters with challans (existing function)
    print("\n📝 STEP 4: Updating TDS Masters with Challan Information...")
    print("-" * 50)
    updated_masters_data = update_tds_masters_with_challans(tds_masters_data, challan_data_list)

    if not updated_masters_data:
        print("❌ Failed to update TDS Masters. Aborting.")
        return None

    print("✅ Successfully updated TDS Masters")

    # Step 5: Generate government template internally
    print("\n📋 STEP 5: Generating Government Template Internally...")
    print("-" * 50)

    try:
        # Import GovernmentTemplateBuilder from Cell 5
        from GovernmentTemplateBuilder import GovernmentTemplateBuilder

        # Create template builder
        builder = GovernmentTemplateBuilder()

        # Create blank template
        govt_template_wb = builder.create_template()

        # Fill the template with data
        filled_template_wb = builder.fill_template(updated_masters_data, challan_data_list)
        print("✅ Using GovernmentTemplateBuilder from Cell 5")

    except ImportError:
        print("⚠️ GovernmentTemplateBuilder not found, creating template manually...")

        # Fallback: Create template manually
        govt_template_wb = Workbook()

        # Remove default sheet
        if 'Sheet' in govt_template_wb.sheetnames:
            govt_template_wb.remove(govt_template_wb['Sheet'])

        # Create required sheets
        ws_deductor = govt_template_wb.create_sheet("DEDUCTOR DETAILS")
        ws_challan = govt_template_wb.create_sheet("CHALLAN DETAILS")
        ws_deductee = govt_template_wb.create_sheet("DEDUCTEE BREAK-UP")

        # Fill with our function that now uses Cell 5's builder
        filled_template_wb = fill_government_template(govt_template_wb, updated_masters_data, challan_data_list)

    # Step 6: Save government template
    output_filename = get_output_filename_from_masters(updated_masters_data)
    govt_template_path = os.path.join(output_folder, output_filename)

    filled_template_wb.save(govt_template_path)
    print(f"✅ Generated government template: {output_filename}")

    # Step 7: Generate reconciliation report
    print("\n📊 STEP 6: Generating Reconciliation Report...")
    print("-" * 50)

    report_path = None
    if validation_results:
        try:
            report = ReconciliationReport(
                validation_results=validation_results,
                masters_data=updated_masters_data,
                challan_data=challan_data_list
            )
            report_filename = f"Reconciliation_{output_filename}"
            report_path = report.generate_report(os.path.join(output_folder, report_filename))
            print(f"✅ Generated reconciliation report: {report_filename}")
        except Exception as e:
            print(f"⚠️ ReconciliationReport not available: {str(e)}")
            print("   Skipping report generation...")

    # Step 8: Final validation (existing function)
    print("\n🔍 STEP 7: Final Validation...")
    print("-" * 50)
    final_validation = validate_tds_totals(updated_masters_data, challan_data_list)
    print("✅ Final Validation: " + ("PASSED" if final_validation else "FAILED"))

    # Return all outputs
    print("\n" + "="*60)
    print("✅ ENHANCED TDS PROCESSING COMPLETE!")
    print("="*60)

    results = {
        'govt_template_path': govt_template_path,
        'reconciliation_report_path': report_path,
        'updated_masters_path': updated_masters_data['file_path'],
        'validation_results': validation_results,
        'final_validation': final_validation,
        'challans_processed': len(challan_data_list),
        'output_filename': output_filename
    }

    print("\n📁 Generated Files:")
    print(f"   1. Government Template: {output_filename}")
    if report_path:
        print(f"   2. Reconciliation Report: {os.path.basename(report_path)}")
    print(f"   3. Updated Masters: {os.path.basename(updated_masters_data['file_path'])}")

    return results

# Backwards compatibility wrapper
def process_tds_returns(pdf_folder_path, masters_file_path, template_file_path=None):
    """
    Wrapper for backwards compatibility - ignores template_file_path
    Uses process_tds_returns_enhanced internally
    """
    if template_file_path:
        print("⚠️ Note: Template file no longer needed - generating internally")

    return process_tds_returns_enhanced(pdf_folder_path, masters_file_path)

print("✅ Enhanced Main Processing Pipeline loaded!")
print("\n📌 Key Features:")
print("   ✅ NO TEMPLATE UPLOAD NEEDED - Generated internally")
print("   ✅ Uses GovernmentTemplateBuilder from Cell 5")
print("   ✅ 20 mandatory fields validation (warn-only)")
print("   ✅ Reconciliation report with validation details")
print("   ✅ All critical business logic preserved")
print("   ✅ Backwards compatible with old function signature")
print("   ✅ FIXED: No forward dependencies - properly imports from Cell 5")
print("\n💡 Usage:")
print("   results = process_tds_returns_enhanced(pdf_folder, masters_file)")
print("   # or for backwards compatibility:")
print("   results = process_tds_returns(pdf_folder, masters_file)")
print("\n📁 Outputs:")
print("   - TDS_Month_Year.xlsx (Government format)")
print("   - Reconciliation_TDS_Month_Year.xlsx (Validation report)")
print("   - TDS_Masters_UPDATED.xlsx (Original with updates)")
print("\n⚠️ Prerequisites:")
print("   - Cell 5 must be run first (contains GovernmentTemplateBuilder)")
print("   - Cell 1-4 for basic functions (extract_all_challans, read_tds_masters, etc.)")

SyntaxError: incomplete input (ipython-input-2506310607.py, line 464)

In [ ]:
# Cell 9: Modified Interactive Interface - 2-File Upload System
# This REPLACES old Cell 7 - Now only requires Masters + PDFs (NO TEMPLATE NEEDED)

import os
import zipfile
import shutil
from google.colab import files
import pandas as pd
from datetime import datetime
import time

# Setup directories
BASE_DIR = '/content/tds_processing'
PDF_DIR = os.path.join(BASE_DIR, 'pdfs')
OUTPUT_FOLDER = os.path.join(BASE_DIR, 'output')

for dir_path in [BASE_DIR, PDF_DIR, OUTPUT_FOLDER]:
    os.makedirs(dir_path, exist_ok=True)

# Global variables to store uploaded files (TEMPLATE REMOVED)
UPLOADED_FILES = {
    'pdfs': [],
    'tds_masters': None
    # 'template': None  # REMOVED - no longer needed
}

def clear_uploads():
    """Clear all uploaded files and reset directories"""
    global UPLOADED_FILES
    UPLOADED_FILES = {'pdfs': [], 'tds_masters': None}  # Template removed
    for dir_path in [PDF_DIR, OUTPUT_FOLDER]:
        if os.path.exists(dir_path):
            shutil.rmtree(dir_path)
            os.makedirs(dir_path, exist_ok=True)
    print("✅ All uploads cleared. Ready for new files.")

def upload_zip_file():
    """Upload and extract a ZIP file containing PDFs and Excel files"""
    print("\n📦 Upload ZIP file containing PDFs and TDS Masters Excel file")
    print("   Note: Template file no longer needed - generated internally!")
    uploaded = files.upload()

    for filename, data in uploaded.items():
        if filename.endswith('.zip'):
            zip_path = os.path.join(BASE_DIR, filename)
            with open(zip_path, 'wb') as f:
                f.write(data)

            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(BASE_DIR)
            os.remove(zip_path)
            print(f"✅ Extracted: {filename}")

            # Process extracted files
            for root, _, files_list in os.walk(BASE_DIR):
                for f in files_list:
                    file_path = os.path.join(root, f)
                    if f.lower().endswith('.pdf'):
                        dest_path = os.path.join(PDF_DIR, f)
                        if not os.path.exists(dest_path):
                            shutil.move(file_path, dest_path)
                            UPLOADED_FILES['pdfs'].append(dest_path)
                    elif f.startswith('TDS_Masters') and f.endswith('.xlsx'):
                        dest_path = os.path.join(BASE_DIR, f)
                        if not os.path.exists(dest_path):
                            shutil.move(file_path, dest_path)
                            UPLOADED_FILES['tds_masters'] = dest_path
                    # Ignore template files - no longer needed
                    elif f.startswith('TDS_Template'):
                        print(f"   ℹ️ Ignoring template file: {f} (no longer needed)")

    print(f"\n✅ ZIP processing complete!")
    print(f"   PDFs: {len(UPLOADED_FILES['pdfs'])} files")
    print(f"   TDS Masters: {os.path.basename(UPLOADED_FILES['tds_masters']) if UPLOADED_FILES['tds_masters'] else 'Not found'}")
    print(f"   Template: Not required - will be generated internally")

def process_uploaded_files():
    """Process uploaded files with enhanced pipeline"""
    if not UPLOADED_FILES['pdfs']:
        print("❌ No PDF files uploaded. Please upload PDFs first.")
        return None

    if not UPLOADED_FILES['tds_masters']:
        print("❌ No TDS Masters file uploaded. Please upload TDS_Masters*.xlsx")
        return None

    # Template check removed - no longer needed

    print("\n✅ All required files uploaded successfully!")
    print("   📄 PDFs: {} files".format(len(UPLOADED_FILES['pdfs'])))
    print("   📊 Masters: {}".format(os.path.basename(UPLOADED_FILES['tds_masters'])))
    print("   📋 Template: Will be generated internally")

    print("\n⏳ Starting enhanced processing...")
    print("="*60)

    # Show sample PDF data
    print("\n📁 Files Ready for Processing:")
    print(f"   PDFs uploaded: {len(UPLOADED_FILES['pdfs'])} files")

    # Display first 4 PDFs with extraction preview
    for pdf in UPLOADED_FILES['pdfs'][:4]:
        print(f"   - {os.path.basename(pdf)}")
        try:
            challan_data = extract_challan_data_from_pdf(pdf)
            print(f"     ✓ Nature of Payment: {challan_data.get('nature_of_payment', 'Not found')}")
            print(f"     ✓ Challan No: {challan_data.get('challan_no', 'Not found')}")
            print(f"     ✓ Tax Amount: ₹{challan_data.get('tax_amount', 'Not found')}")
        except:
            print(f"     ⚠️ Preview not available")

    if len(UPLOADED_FILES['pdfs']) > 4:
        print(f"   ... and {len(UPLOADED_FILES['pdfs']) - 4} more PDFs")

    print(f"\n   TDS Masters: {os.path.basename(UPLOADED_FILES['tds_masters'])}")
    print("="*60)

    print("\n🚀 Starting Enhanced TDS Processing...")
    print("\n✨ ENHANCEMENTS:")
    print("   ✅ Government template generated internally (no upload needed)")
    print("   ✅ 20 mandatory fields validation")
    print("   ✅ Reconciliation report with validation details")
    print("   ✅ Enhanced error handling and reporting")

    print("\n🔧 PROCESSING FEATURES:")
    print("   ✓ Using openpyxl to read values directly")
    print("   ✓ BSR codes padded to 7 digits with leading zeros")
    print("   ✓ Challan numbers preserved as strings")
    print("   ✓ Deduplicating challans by challan number")
    print("   ✓ Rounding amounts UP to nearest rupee")
    print("   ✓ Dates properly parsed and formatted")
    print("   ✓ Mode of payment converted to uppercase")
    print("   ✓ Rate formatting fixed (0.1 → 10%)")

    # Run the enhanced process (NO TEMPLATE PARAMETER)
    try:
        results = process_tds_returns_enhanced(
            pdf_folder_path=PDF_DIR,
            masters_file_path=UPLOADED_FILES['tds_masters'],
            output_folder=OUTPUT_FOLDER
        )

        if results:
            print("\n" + "="*60)
            print("📊 PROCESSING SUMMARY")
            print("="*60)
            print(f"✅ Unique Challans Processed: {results['challans_processed']}")
            print(f"✅ Government Template Generated: {results['output_filename']}")
            print(f"✅ TDS Masters Updated: {os.path.basename(results['updated_masters_path'])}")

            if results.get('reconciliation_report_path'):
                print(f"✅ Reconciliation Report Created: {os.path.basename(results['reconciliation_report_path'])}")

            if results.get('validation_results'):
                val_summary = results['validation_results']['summary']
                print(f"\n📋 Validation Summary:")
                print(f"   - Mandatory Fields: {val_summary['fields_present']}/20 present")
                print(f"   - Validation Status: {val_summary['validation_status']}")

            print(f"\n✅ Final Validation: {'PASSED' if results['final_validation'] else 'FAILED'}")
            print("="*60)

            print("\n✅ TDS Return Processing Complete!")

            # Download files
            print("\n📥 DOWNLOADING OUTPUT FILES...")
            print("-"*40)

            # Download government template
            if os.path.exists(results['govt_template_path']):
                files.download(results['govt_template_path'])
                print(f"✅ Downloaded: {os.path.basename(results['govt_template_path'])}")

            # Download reconciliation report if exists
            if results.get('reconciliation_report_path') and os.path.exists(results['reconciliation_report_path']):
                files.download(results['reconciliation_report_path'])
                print(f"✅ Downloaded: {os.path.basename(results['reconciliation_report_path'])}")

            # Download updated masters
            if os.path.exists(results['updated_masters_path']):
                files.download(results['updated_masters_path'])
                print(f"✅ Downloaded: {os.path.basename(results['updated_masters_path'])}")

            print("\n✅ All files processed and downloaded successfully!")
            print("📋 Check your browser's download folder for the output files.")

            return results
        else:
            print("\n❌ Processing failed. Please check the error messages above.")
            return None

    except Exception as e:
        print(f"\n❌ Error during processing: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

def start_interactive_upload():
    """Start the interactive upload process - NOW ONLY 2 FILES"""
    clear_uploads()

    print("\n🎯 ENHANCED TDS PROCESSING - 2-FILE SYSTEM")
    print("="*60)
    print("No template upload needed - generated internally!")
    print("="*60)

    # Step 1: Upload PDFs
    print("\n📋 STEP 1: Upload PDF Files")
    print("-"*40)
    print("Click 'Choose Files' and select your PDF challan files")
    uploaded = files.upload()

    for filename, data in uploaded.items():
        if filename.lower().endswith('.pdf'):
            file_path = os.path.join(PDF_DIR, filename)
            with open(file_path, 'wb') as f:
                f.write(data)
            UPLOADED_FILES['pdfs'].append(file_path)
            print(f"✅ Uploaded: {filename}")

    if not UPLOADED_FILES['pdfs']:
        print("⚠️ No PDF files uploaded. Please run this cell again.")
        return None

    print(f"\n✅ Uploaded {len(UPLOADED_FILES['pdfs'])} PDF files")

    # Step 2: Upload TDS Masters
    print("\n📋 STEP 2: Upload TDS Masters File")
    print("-"*40)
    print("Click 'Choose Files' and select your 'TDS_Masters*.xlsx' file")
    uploaded = files.upload()

    for filename, data in uploaded.items():
        if filename.endswith('.xlsx'):
            file_path = os.path.join(BASE_DIR, filename)
            with open(file_path, 'wb') as f:
                f.write(data)
            UPLOADED_FILES['tds_masters'] = file_path
            print(f"✅ Uploaded: {filename}")
            break  # Only take first Excel file

    if not UPLOADED_FILES['tds_masters']:
        print("⚠️ No TDS Masters file uploaded. Please run this cell again.")
        return None

    # Step 3: REMOVED - Template no longer needed
    print("\n📋 STEP 3: Generate Government Template")
    print("-"*40)
    print("✅ Template will be generated internally - no upload needed!")

    # Process files
    print("\n" + "="*60)
    results = process_uploaded_files()

    if results:
        print("\n✅ SUCCESS! Processing complete.")
    else:
        print("\n⚠️ Processing encountered issues. Please review messages above.")

    return results

def download_tds_masters_template():
    """Download blank TDS_Masters template for users who need it"""
    print("\n📥 Generating TDS_Masters Template...")
    try:
        # Check if TDSMastersTemplateGenerator exists
        generator = TDSMastersTemplateGenerator()
        template_wb = generator.create_template()
        template_path = os.path.join(BASE_DIR, 'TDS_Masters_Template.xlsx')
        template_wb.save(template_path)

        files.download(template_path)
        print("✅ Downloaded: TDS_Masters_Template.xlsx")
        print("\n📝 Instructions:")
        print("   1. Fill in all 20 mandatory fields in INPUT-DEDUCTOR DETAILS")
        print("   2. Add party details in TDS PARTIES sheet")
        print("   3. Save and upload the filled file for processing")
    except Exception as e:
        print(f"⚠️ Template generator not available: {str(e)}")
        print("   Please use your existing TDS_Masters file")

# Main execution options
print("\n" + "="*60)
print("🎯 ENHANCED TDS AUTOMATION - 2-FILE SYSTEM")
print("="*60)
print("\n✨ KEY IMPROVEMENTS:")
print("   • No template upload needed (generated internally)")
print("   • Only 2 files required: Masters + PDFs")
print("   • 20 mandatory fields validation")
print("   • Enhanced reconciliation report")
print("\n📌 QUICK START OPTIONS:")
print("-"*40)
print("\n1️⃣ STANDARD UPLOAD (Recommended):")
print("   Run: start_interactive_upload()")
print("\n2️⃣ ZIP FILE UPLOAD:")
print("   Run: upload_zip_file()")
print("   Then: process_uploaded_files()")
print("\n3️⃣ DOWNLOAD BLANK TEMPLATE:")
print("   Run: download_tds_masters_template()")
print("\n4️⃣ CLEAR & START FRESH:")
print("   Run: clear_uploads()")
print("\n" + "="*60)

# Auto-start option (comment out if not wanted)
print("\n🚀 Ready to start? Run: start_interactive_upload()")

In [ ]:
# Cell 6: Testing individual components and troubleshooting - FIXED VERSION
# Use these functions to test each step separately if you encounter issues

def test_pdf_extraction_single(pdf_path):
    """Test extraction from a single PDF file"""
    print(f"🧪 Testing PDF extraction for: {pdf_path}")
    print("-" * 50)

    data = extract_challan_data_from_pdf(pdf_path)

    print("\n📊 Extracted Data:")
    for key, value in data.items():
        if key != 'error':
            print(f"   {key}: {value}")

    return data

def test_pdf_deduplication(pdf_folder):
    """Test PDF extraction with deduplication"""
    print(f"🧪 Testing PDF extraction with deduplication for: {pdf_folder}")
    print("-" * 50)

    # Get all PDF files
    pdf_files = [f for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]
    print(f"Total PDF files found: {len(pdf_files)}")

    # Extract without deduplication to show duplicates
    all_challans = []
    challan_numbers = []

    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder, pdf_file)
        data = extract_challan_data_from_pdf(pdf_path)
        if data.get('challan_no'):
            all_challans.append(data)
            challan_numbers.append(data.get('challan_no'))

    # Count duplicates
    from collections import Counter
    challan_counts = Counter(challan_numbers)

    print(f"\n📊 Challan Analysis:")
    print(f"   Total PDFs processed: {len(pdf_files)}")
    print(f"   Total challans extracted: {len(all_challans)}")
    print(f"   Unique challan numbers: {len(set(challan_numbers))}")

    print("\n📊 Challan Number Frequency:")
    for challan_no, count in challan_counts.most_common():
        if count > 1:
            print(f"   Challan {challan_no}: appears {count} times (DUPLICATE)")
        else:
            print(f"   Challan {challan_no}: appears {count} time")

    # Now test with deduplication
    print("\n🧪 Testing with deduplication...")
    deduplicated_challans = extract_all_challans(pdf_folder)

    print(f"\n✅ After deduplication: {len(deduplicated_challans)} unique challans")

    return deduplicated_challans

def test_tds_masters_reading(file_path):
    """Test reading TDS Masters file"""
    print(f"🧪 Testing TDS Masters reading: {file_path}")
    print("-" * 50)

    data = read_tds_masters(file_path)

    if data:
        print("\n📊 Sheet Summary:")
        print(f"   TDS CODES: {len(data['tds_codes'])} entries")
        print(f"   TDS PARTIES: {len(data['tds_parties'])} parties")
        print(f"   Challan Details: {len(data['challan_details'])} challans")
        print(f"   TDS RATES: {len(data['tds_rates'])} rates")

        # Show column code mapping
        print("\n📊 Column Code Mapping Found:")
        code_to_column_name = data.get('code_to_column_name', {})
        for code, col_name in sorted(code_to_column_name.items())[:20]:  # Show first 20
            print(f"   {code} → {col_name}")

        # Check for critical columns
        critical_codes = ['(415)', '(415A)', '(416)', '(417)', '(418)', '(419)', '(421)', '(427)']
        missing_codes = [code for code in critical_codes if code not in code_to_column_name]

        if missing_codes:
            print("\n⚠️  Missing critical column codes:")
            for code in missing_codes:
                print(f"   - {code}")
        else:
            print("\n✅ All critical column codes found!")

        # Show sample party data
        print("\n📊 Sample TDS PARTIES data (first 3 rows):")
        print(data['tds_parties'].head(3))

    return data

def test_date_extraction(tds_masters_data):
    """Test date extraction for output filename"""
    print("🧪 Testing date extraction for output filename...")
    print("-" * 50)

    output_filename = get_output_filename_from_masters(tds_masters_data)
    print(f"\n📊 Generated output filename: {output_filename}")

    # Show the dates found
    code_to_column_name = tds_masters_data.get('code_to_column_name', {})
    date_col = code_to_column_name.get('(418)')  # Date of Payment/credit

    if date_col and date_col in tds_masters_data['tds_parties'].columns:
        dates = tds_masters_data['tds_parties'][date_col].dropna()
        if not dates.empty:
            print(f"\n📊 Payment dates found:")
            for i, date in enumerate(dates.head(5)):
                print(f"   Row {i}: {date}")

            first_date = pd.to_datetime(dates.iloc[0])
            print(f"\n📊 Using first date: {first_date.strftime('%d/%m/%Y')}")
            print(f"   Month: {first_date.strftime('%B')}")
            print(f"   Year: {first_date.strftime('%Y')}")

def create_test_summary(tds_masters_data, challan_data_list):
    """Create a summary report of the data before processing"""
    print("\n📊 PRE-PROCESSING SUMMARY REPORT")
    print("=" * 60)

    # Get column mappings
    code_to_column_name = tds_masters_data.get('code_to_column_name', {})

    # Find required columns by codes
    col_type_payment = code_to_column_name.get('(415A)', None)
    col_tds = code_to_column_name.get('(421)', None)

    if not col_type_payment or not col_tds:
        print("⚠️  Warning: Could not find required columns by codes")
        print(f"   Payment Type (415A): {col_type_payment}")
        print(f"   TDS Amount (421): {col_tds}")
        return

    # Challan summary (now with unique challans only)
    print("\n1️⃣ UNIQUE CHALLAN SUMMARY:")
    print("-" * 40)
    challan_summary = {}
    total_tax = 0

    for challan in challan_data_list:
        nop = challan.get('nature_of_payment', 'Unknown')
        tax = float(challan.get('tax_amount', 0))
        # Round UP
        tax = math.ceil(tax) if tax else 0

        if nop not in challan_summary:
            challan_summary[nop] = {'count': 0, 'total': 0, 'challan_numbers': []}

        challan_summary[nop]['count'] += 1
        challan_summary[nop]['total'] += tax
        challan_summary[nop]['challan_numbers'].append(challan.get('challan_no', 'N/A'))
        total_tax += tax

    for nop, data in sorted(challan_summary.items()):
        print(f"   {nop}: {data['count']} unique challan(s), Total: ₹{data['total']:,.0f}")
        print(f"      Challan Numbers: {', '.join(data['challan_numbers'])}")
    print(f"   TOTAL TAX: ₹{total_tax:,.0f}")

    # Party summary
    print("\n2️⃣ PARTY SUMMARY:")
    print("-" * 40)
    tds_parties = tds_masters_data['tds_parties']
    party_summary = {}

    for _, party in tds_parties.iterrows():
        payment_type = str(party.get(col_type_payment, '')).strip() if col_type_payment else ''
        tds = float(party.get(col_tds, 0)) if col_tds and pd.notna(party.get(col_tds)) else 0
        # Round UP
        tds = math.ceil(tds) if tds else 0

        if payment_type and payment_type != 'nan':
            if payment_type not in party_summary:
                party_summary[payment_type] = {'count': 0, 'total': 0}

            party_summary[payment_type]['count'] += 1
            party_summary[payment_type]['total'] += tds

    for payment_type, data in sorted(party_summary.items()):
        print(f"   {payment_type}: {data['count']} parties, Total TDS: ₹{data['total']:,.0f}")

    # Matching preview
    print("\n3️⃣ MATCHING PREVIEW:")
    print("-" * 40)
    for nop in challan_summary:
        challan_total = challan_summary[nop]['total']
        party_total = party_summary.get(nop.replace(' ', ''), {}).get('total', 0)
        diff = abs(challan_total - party_total)

        if diff <= 1:
            status = "✅"
        else:
            status = "❌"

        print(f"   {nop}: Challan ₹{challan_total:,.0f} vs Party ₹{party_total:,.0f} {status}")

    print("=" * 60)

# Troubleshooting tips - UPDATED
print("🔧 TROUBLESHOOTING TIPS - FIXED VERSION")
print("=" * 60)
print("\n1. PDF Extraction Issues:")
print("   - Ensure PDFs are text-based (not scanned images)")
print("   - Check if all PDFs are in ITNS 281 format")
print("   - Try testing a single PDF first")
print("   - BSR codes and challan numbers are now preserved as strings ✅")

print("\n2. Deduplication Issues:")
print("   - Check if duplicate PDFs have same challan numbers")
print("   - Use test_pdf_deduplication() to see duplicate analysis")
print("   - Ensure challan numbers are being extracted correctly")
print("   - Look for file names with (1), (2) suffixes - these are duplicates")

print("\n3. Excel File Issues:")
print("   - Ensure Excel files are not password protected")
print("   - Close Excel files before running the script")
print("   - Check if sheet names match exactly")
print("   - Verify column codes are in row 2 (can be (code) or -code format)")

print("\n4. Column Code Issues FIXED:")
print("   - ✅ Now handles both (415A) and -415A formats")
print("   - ✅ Falls back to column name matching if codes not found")
print("   - ✅ Maps all critical columns including those with - prefix")
print("   - Codes are normalized to (code) format internally")

print("\n5. NaN Issue FIXED:")
print("   - ✅ Uses openpyxl to read values directly, avoiding pandas type inference")
print("   - ✅ TDS values no longer become NaN after save/reload")
print("   - ✅ All numeric values preserved correctly throughout processing")

print("\n6. Date Format Issues FIXED:")
print("   - ✅ Dates are parsed from strings to datetime objects")
print("   - ✅ Excel date format is set explicitly to DD/MM/YYYY")
print("   - ✅ Prevents data corruption during save/reload")

print("\n7. Rate Formatting FIXED:")
print("   - ✅ Decimal rates (0.1) are multiplied by 100 to get percentage (10%)")
print("   - ✅ Rates are formatted consistently with % symbol")
print("   - ✅ Handles both decimal and percentage input formats")

print("\n8. Leading Zeros FIXED:")
print("   - ✅ BSR codes padded to 7 digits (e.g., 240020 → 0240020)")
print("   - ✅ Challan numbers preserved as strings (e.g., 03636 stays 03636)")
print("   - ✅ Written as strings to Excel to prevent conversion to numbers")

print("\n9. Validation Issues:")
print("   - Check if Nature of Payment codes match (94A vs 94 A)")
print("   - Verify TDS amounts in Masters match challan amounts")
print("   - Look for rounding differences (we round UP)")
print("   - Final validation should now pass with fixed data preservation ✅")

print("\n10. Common Error Messages:")
print("   - 'Permission denied': Close the Excel file")
print("   - 'Sheet not found': Check sheet names in Excel")
print("   - 'No PDF files found': Check PDF folder path")
print("   - 'KeyError': Column code not found (now has fallbacks)")

print("\n11. Expected File Names (UPDATED):")
print("   - Masters: Should start with 'TDS_Masters'")
print("   - Template: Should start with 'TDS_Template'")
print("   - Output: Will be 'TDS_[Month]_[Year].xlsx'")
print("=" * 60)

print("\n💡 Test individual components using:")
print("   test_pdf_extraction_single('path/to/single.pdf')")
print("   test_pdf_deduplication('path/to/pdf/folder')")
print("   test_tds_masters_reading('path/to/TDS_Masters.xlsx')")
print("   test_date_extraction(tds_masters_data)")
print("   create_test_summary(tds_masters_data, challan_data_list)")

print("\n🎯 KEY FIXES IMPLEMENTED:")
print("   ✅ NaN issue resolved with openpyxl direct reading")
print("   ✅ Leading zeros preserved for BSR codes and challan numbers")
print("   ✅ Rate formatting fixed (0.1 → 10%)")
print("   ✅ Column codes with - prefix properly handled")
print("   ✅ Fallback to column name matching for all fields")
print("   ✅ Dates parsed and formatted correctly")
print("   ✅ Mode of payment converted to uppercase")
print("   ✅ Data integrity preserved throughout processing")

In [ ]:
# Cell 10: Main Execution Cell - Simple Interface
# Run this cell to start the enhanced TDS processing system

print("╔" + "═"*58 + "╗")
print("║" + " "*15 + "TDS AUTOMATION SYSTEM 2.0" + " "*18 + "║")
print("║" + " "*12 + "Enhanced 2-File Processing System" + " "*13 + "║")
print("╚" + "═"*58 + "╝")

print("\n📋 SYSTEM STATUS:")
print("   ✅ Government template: Generated internally")
print("   ✅ Files required: Only 2 (Masters + PDFs)")
print("   ✅ Validation: 20 mandatory fields check")
print("   ✅ Reports: Reconciliation with validation details")

print("\n" + "─"*60)
print("🚀 QUICK START MENU")
print("─"*60)

print("\n1️⃣  PROCESS TDS RETURNS (Standard Upload)")
print("    >>> start_interactive_upload()")

print("\n2️⃣  UPLOAD ZIP FILE (Batch Processing)")
print("    >>> upload_zip_file()")
print("    >>> process_uploaded_files()")

print("\n3️⃣  DOWNLOAD BLANK TDS_MASTERS TEMPLATE")
print("    >>> download_tds_masters_template()")

print("\n4️⃣  CLEAR ALL & START FRESH")
print("    >>> clear_uploads()")

print("\n5️⃣  TEST INDIVIDUAL COMPONENTS")
print("    >>> test_pdf_extraction_single('/path/to/pdf')")
print("    >>> test_tds_masters_reading('/path/to/masters.xlsx')")
print("    >>> test_validation('/path/to/masters.xlsx')")

print("\n" + "─"*60)
print("📁 FILE LOCATIONS")
print("─"*60)
print(f"   • Base Directory: {BASE_DIR}")
print(f"   • PDFs Directory: {PDF_DIR}")
print(f"   • Output Directory: {OUTPUT_FOLDER}")

print("\n" + "─"*60)
print("📊 CURRENT STATUS")
print("─"*60)

# Check current uploads
pdf_count = len(UPLOADED_FILES.get('pdfs', []))
masters_status = "✅ Uploaded" if UPLOADED_FILES.get('tds_masters') else "❌ Not uploaded"

print(f"   • PDF Files: {pdf_count} files uploaded")
print(f"   • TDS Masters: {masters_status}")
if UPLOADED_FILES.get('tds_masters'):
    print(f"     File: {os.path.basename(UPLOADED_FILES['tds_masters'])}")

# Check for existing output files
if os.path.exists(OUTPUT_FOLDER):
    output_files = [f for f in os.listdir(OUTPUT_FOLDER) if f.endswith('.xlsx')]
    if output_files:
        print(f"\n   • Output Files Available: {len(output_files)}")
        for f in output_files[:3]:
            print(f"     - {f}")

print("\n" + "═"*60)
print("📌 TO BEGIN: Run >>> start_interactive_upload()")
print("═"*60)

# Quick start function for immediate execution
def quick_start():
    """Quick start the TDS processing"""
    print("\n🎯 Starting TDS Processing System...")
    return start_interactive_upload()

# Auto-execution prompt
print("\n💡 TIP: For immediate start, run >>> quick_start()")